<a href="https://colab.research.google.com/github/zbatzioutilburg/2ndThesisSubmission/blob/main/2ndDataProcessingMethodology2ndSubmissionCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -q -U keras-tuner

     |████████████████████████████████| 98 kB 3.2 MB/s 


In [2]:
pip install PyPortfolioOpt

     |████████████████████████████████| 61 kB 2.9 MB/s 
     |████████████████████████████████| 2.7 MB 10.1 MB/s 
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


In [3]:
import math
import pandas as pd
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import random
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from functools import reduce
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
import pypfopt
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [4]:
random.seed(1)

In [6]:
NAESX = pd.read_csv('NAESX.csv', sep = ',', index_col = 0)
NAESX.index = pd.to_datetime(NAESX.index)
NAESX = NAESX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
NAESX.columns = ['NAESX Adj. Close']
NAESX = np.log(NAESX['NAESX Adj. Close']) - np.log(NAESX['NAESX Adj. Close'].shift(1))

VBMFX = pd.read_csv('VBMFX.csv', sep = ',', index_col = 0)
VBMFX.index = pd.to_datetime(VBMFX.index)
VBMFX = VBMFX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
VBMFX.columns = ['VBMFX Adj. Close']
VBMFX = np.log(VBMFX['VBMFX Adj. Close']) - np.log(VBMFX['VBMFX Adj. Close'].shift(1))

VEIEX = pd.read_csv('VEIEX.csv', sep = ',', index_col = 0)
VEIEX.index = pd.to_datetime(VEIEX.index)
VEIEX = VEIEX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
VEIEX.columns = ['VEIEX Adj. Close']
VEIEX = np.log(VEIEX['VEIEX Adj. Close']) - np.log(VEIEX['VEIEX Adj. Close'].shift(1))

VFINX = pd.read_csv('VFINX.csv', sep = ',', index_col = 0)
VFINX.index = pd.to_datetime(VFINX.index)
VFINX = VFINX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
VFINX.columns = ['VFINX Adj. Close']
VFINX = np.log(VFINX['VFINX Adj. Close']) - np.log(VFINX['VFINX Adj. Close'].shift(1))

VGSLX = pd.read_csv('VGSLX.csv', sep = ',', index_col = 0)
VGSLX.index = pd.to_datetime(VGSLX.index)
VGSLX = VGSLX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
VGSLX.columns = ['VGSLX Adj. Close']
VGSLX = np.log(VGSLX['VGSLX Adj. Close']) - np.log(VGSLX['VGSLX Adj. Close'].shift(1))

VGTSX = pd.read_csv('VGTSX.csv', sep = ',', index_col = 0)
VGTSX.index = pd.to_datetime(VGTSX.index)
VGTSX = VGTSX.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
VGTSX.columns = ['VGTSX Adj. Close']
VGTSX = np.log(VGTSX['VGTSX Adj. Close']) - np.log(VGTSX['VGTSX Adj. Close'].shift(1))

In [ ]:
T5YIE = pd.read_csv('T5YIE.csv', sep = ',', index_col = 0)
T5YIE.index = pd.to_datetime(T5YIE.index)
T5YIE.T5YIE = T5YIE.T5YIE.replace('.', '')
T5YIE.T5YIE = pd.to_numeric(T5YIE.T5YIE, errors = 'coerce')
T5YIE.T5YIE = T5YIE.T5YIE.ffill()
T5YIE.T5YIE = np.log(T5YIE.T5YIE) - np.log(T5YIE.T5YIE.shift(1))

DTB3 = pd.read_csv('DTB3.csv', sep = ',', index_col = 0)
DTB3.index = pd.to_datetime(DTB3.index)
DTB3.DTB3 = DTB3.DTB3.replace('.', '')
DTB3.DTB3 = pd.to_numeric(DTB3.DTB3, errors = 'coerce')
DTB3.DTB3 = DTB3.DTB3.ffill()
DTB3.DTB3 = np.log(DTB3.DTB3) - np.log(DTB3.DTB3.shift(1))

DTWEXM = pd.read_csv('DTWEXM.csv', sep = ',', index_col = 0)
DTWEXM.index = pd.to_datetime(DTWEXM.index)
DTWEXM.DTWEXM = DTWEXM.DTWEXM.replace('.', '')
DTWEXM.DTWEXM = pd.to_numeric(DTWEXM.DTWEXM, errors = 'coerce')
DTWEXM.DTWEXM = DTWEXM.DTWEXM.ffill()
DTWEXM.DTWEXM = np.log(DTWEXM.DTWEXM) - np.log(DTWEXM.DTWEXM.shift(1))

USEPUINDXD = pd.read_csv('USEPUINDXD.csv', sep = ',', index_col = 0)
USEPUINDXD.index = pd.to_datetime(USEPUINDXD.index)
USEPUINDXD.USEPUINDXD = USEPUINDXD.USEPUINDXD.replace('.', '')
USEPUINDXD.USEPUINDXD = pd.to_numeric(USEPUINDXD.USEPUINDXD, errors = 'coerce')
USEPUINDXD.USEPUINDXD = USEPUINDXD.USEPUINDXD.ffill()
USEPUINDXD.USEPUINDXD = np.log(USEPUINDXD.USEPUINDXD) - np.log(USEPUINDXD.USEPUINDXD.shift(1))

DCOILWTICO = pd.read_csv('DCOILWTICO.csv', sep = ',', index_col = 0)
DCOILWTICO.index = pd.to_datetime(DCOILWTICO.index)
DCOILWTICO.DCOILWTICO = DCOILWTICO.DCOILWTICO.replace('.', '')
DCOILWTICO.DCOILWTICO = pd.to_numeric(DCOILWTICO.DCOILWTICO, errors = 'coerce')
DCOILWTICO.DCOILWTICO = DCOILWTICO.DCOILWTICO.ffill()
DCOILWTICO.DCOILWTICO = np.log(DCOILWTICO.DCOILWTICO) - np.log(DCOILWTICO.DCOILWTICO.shift(1))

USD1MTD156N = pd.read_csv('USD1MTD156N.csv', sep = ',', index_col = 0)
USD1MTD156N.index = pd.to_datetime(USD1MTD156N.index)
USD1MTD156N.USD1MTD156N = USD1MTD156N.USD1MTD156N.replace('.', '')
USD1MTD156N.USD1MTD156N = pd.to_numeric(USD1MTD156N.USD1MTD156N, errors = 'coerce')
USD1MTD156N.USD1MTD156N = USD1MTD156N.USD1MTD156N.ffill()
USD1MTD156N.USD1MTD156N = np.log(USD1MTD156N.USD1MTD156N) - np.log(USD1MTD156N.USD1MTD156N.shift(1))

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
macro = [USEPUINDXD, USD1MTD156N, DTWEXM, T5YIE, DTB3, DCOILWTICO]
X = reduce(lambda left, right : pd.merge(left, right,
                                         left_index = True, right_index = True,
                                         how = 'outer'), macro)
X

,USEPUINDXD,USD1MTD156N,DTWEXM,T5YIE,DTB3,DCOILWTICO
DATE,,,,,,
2003-01-01,NaN,NaN,NaN,NaN,NaN,NaN
2003-01-02,0.295029,NaN,0.006191,NaN,NaN,NaN
2003-01-03,0.096997,0.000000,-0.004300,-0.015504,0.000000,0.039558
2003-01-04,-0.641239,NaN,NaN,NaN,NaN,NaN
2003-01-05,0.748111,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2020-12-27,-0.667527,NaN,NaN,NaN,NaN,NaN
2020-12-28,-0.501178,0.000000,NaN,0.000000,0.200671,-0.014214
2020-12-29,0.275189,0.011101,NaN,-0.010471,-0.095310,0.007341


In [ ]:
X.DTB3 = X.DTB3.ffill()
X.T5YIE = X.T5YIE.ffill()
X.USEPUINDXD = X.USEPUINDXD.ffill()
X.USD1MTD156N = X.USD1MTD156N.ffill()
X.DTWEXM = X.DTWEXM.ffill()
X.DCOILWTICO = X.DCOILWTICO.ffill()

In [ ]:
X.DTB3.replace([np.inf, -np.inf], [100, -100], inplace=True)
X.T5YIE.replace([np.inf, -np.inf], [100, -100], inplace=True)

In [ ]:
X = X['2003-01-03':'2019-12-30']

In [ ]:
X.describe()

,USEPUINDXD,USD1MTD156N,DTWEXM,T5YIE,DTB3,DCOILWTICO
count,6206.000000,6206.000000,6206.000000,6206.000000,6206.000000,6206.000000
mean,-0.000188,0.000039,0.000033,0.024025,0.012343,0.000379
std,0.572021,0.013241,0.004538,1.272022,7.930517,0.022838
min,-3.483855,-0.418993,-0.041066,-0.854415,-100.000000,-0.151909
25%,-0.343347,-0.001345,-0.002476,-0.011050,-0.007905,-0.011081
50%,-0.008362,0.000000,0.000000,0.000000,0.000000,0.000122
75%,0.345597,0.002283,0.002438,0.011332,0.007299,0.011803
max,3.892211,0.283306,0.024859,100.000000,100.000000,0.164137


In [7]:
full_index = pd.date_range(start = '2003-01-03', end = '2019-12-30')
full_index = pd.Series(full_index, name = 'Full Index')
full_index.index = full_index

In [8]:
y1 = pd.merge(full_index, NAESX,
                     left_index = True, right_index = True,
                     how = 'left')

y2 = pd.merge(full_index, VBMFX,
                     left_index = True, right_index = True,
                     how = 'left')

y3 = pd.merge(full_index, VEIEX,
                     left_index = True, right_index = True,
                     how = 'left')

y4 = pd.merge(full_index, VFINX,
                     left_index = True, right_index = True,
                     how = 'left')

y5 = pd.merge(full_index, VGSLX,
                     left_index = True, right_index = True,
                     how = 'left')

y6 = pd.merge(full_index, VGTSX,
                     left_index = True, right_index = True,
                     how = 'left')

In [9]:
y1['NAESX Adj. Close'].fillna(method = 'ffill', inplace = True)
y2['VBMFX Adj. Close'].fillna(method = 'ffill', inplace = True)
y3['VEIEX Adj. Close'].fillna(method = 'ffill', inplace = True)
y4['VFINX Adj. Close'].fillna(method = 'ffill', inplace = True)
y5['VGSLX Adj. Close'].fillna(method = 'ffill', inplace = True)
y6['VGTSX Adj. Close'].fillna(method = 'ffill', inplace = True)
y1.drop(columns = ['Full Index'], inplace = True)
y2.drop(columns = ['Full Index'], inplace = True)
y3.drop(columns = ['Full Index'], inplace = True)
y4.drop(columns = ['Full Index'], inplace = True)
y5.drop(columns = ['Full Index'], inplace = True)
y6.drop(columns = ['Full Index'], inplace = True)

In [10]:
#X_train = X[0 : int(0.7*len(X))]
#X_val = X[int(0.7*len(X)) : int(0.85*len(X))]
#X_test = X[int(0.85*len(X)) : ]

y1_train = y1[0: int(0.7*len(y1))]
y1_val = y1[int(0.7*len(y1)) : int(0.85*len(y1))]
y1_test = y1[int(0.85*len(y1)) : ]

y2_train = y2[0: int(0.7*len(y2))]
y2_val = y2[int(0.7*len(y2)) : int(0.85*len(y2))]
y2_test = y2[int(0.85*len(y2)) : ]

y3_train = y1[0: int(0.7*len(y3))]
y3_val = y3[int(0.7*len(y3)) : int(0.85*len(y3))]
y3_test = y3[int(0.85*len(y3)) : ]

y4_train = y4[0: int(0.7*len(y4))]
y4_val = y4[int(0.7*len(y4)) : int(0.85*len(y4))]
y4_test = y4[int(0.85*len(y4)) : ]

y5_train = y5[0: int(0.7*len(y5))]
y5_val = y5[int(0.7*len(y5)) : int(0.85*len(y5))]
y5_test = y5[int(0.85*len(y5)) : ]

y6_train = y6[0: int(0.7*len(y6))]
y6_val = y6[int(0.7*len(y6)) : int(0.85*len(y6))]
y6_test = y6[int(0.85*len(y6)) : ]

In [ ]:
plot_index = X.index
plot_index

DatetimeIndex(['2003-01-03', '2003-01-04', '2003-01-05', '2003-01-06',
               '2003-01-07', '2003-01-08', '2003-01-09', '2003-01-10',
               '2003-01-11', '2003-01-12',
               ...
               '2019-12-21', '2019-12-22', '2019-12-23', '2019-12-24',
               '2019-12-25', '2019-12-26', '2019-12-27', '2019-12-28',
               '2019-12-29', '2019-12-30'],
              dtype='datetime64[ns]', name='DATE', length=6206, freq=None)

In [11]:
y1_test.describe()

,NAESX Adj. Close
count,931.000000
mean,0.000617
std,0.009324
min,-0.039093
25%,-0.003375
50%,0.001435
75%,0.005997
max,0.048059


In [12]:
y2_test.describe()

,VBMFX Adj. Close
count,931.000000
mean,0.000192
std,0.001985
min,-0.006355
25%,-0.000955
50%,0.000000
75%,0.001406
max,0.007293


In [13]:
y3_test.describe()

,VEIEX Adj. Close
count,931.000000
mean,0.000579
std,0.008493
min,-0.033795
25%,-0.003608
50%,0.001768
75%,0.005883
max,0.029335


In [14]:
y4_test.describe()

,VFINX Adj. Close
count,931.000000
mean,0.000672
std,0.008667
min,-0.041920
25%,-0.002140
50%,0.000993
75%,0.005091
max,0.048399


In [15]:
y5_test.describe()

,VGSLX Adj. Close
count,931.000000
mean,0.000448
std,0.008265
min,-0.038258
25%,-0.003576
50%,0.001273
75%,0.005556
max,0.033980


In [16]:
y6_test.describe()

,VGTSX Adj. Close
count,931.000000
mean,0.000518
std,0.007076
min,-0.035035
25%,-0.003296
50%,0.001154
75%,0.005042
max,0.028949


In [ ]:
X_train = X_train.values
X_val = X_val.values
X_test = X_test.values

y1_train = y1_train.values
y1_val = y1_val.values
y1_test = y1_test.values

y2_train = y2_train.values
y2_val = y2_val.values
y2_test = y2_test.values

y3_train = y3_train.values
y3_val = y3_val.values
y3_test = y3_test.values

y4_train = y4_train.values
y4_val = y4_val.values
y4_test = y4_test.values

y5_train = y5_train.values
y5_val = y5_val.values
y5_test = y5_test.values

y6_train = y6_train.values
y6_val = y6_val.values
y6_test = y6_test.values

In [ ]:
y1_train = y1_train.reshape(len(y1_train), 1)
y1_val = y1_val.reshape(len(y1_val), 1)
y1_test = y1_test.reshape(len(y1_test), 1)

y2_train = y2_train.reshape(len(y2_train), 1)
y2_val = y2_val.reshape(len(y2_val), 1)
y2_test = y2_test.reshape(len(y2_test), 1)

y3_train = y3_train.reshape(len(y3_train), 1)
y3_val = y3_val.reshape(len(y3_val), 1)
y3_test = y3_test.reshape(len(y3_test), 1)

y4_train = y4_train.reshape(len(y4_train), 1)
y4_val = y4_val.reshape(len(y4_val), 1)
y4_test = y4_test.reshape(len(y4_test), 1)

y5_train = y5_train.reshape(len(y5_train), 1)
y5_val = y5_val.reshape(len(y5_val), 1)
y5_test = y5_test.reshape(len(y5_test), 1)

y6_train = y6_train.reshape(len(y6_train), 1)
y6_val = y6_val.reshape(len(y6_val), 1)
y6_test = y6_test.reshape(len(y6_test), 1)

In [ ]:
X_train = (X_train - X_train.mean()) / X_train.std()
X_val = (X_val - X_train.mean()) / X_train.std()
X_test = (X_test - X_train.mean()) / X_train.std()

In [ ]:
baseline_1 = LinearRegression().fit(X_train, y1_train)
baseline_2 = LinearRegression().fit(X_train, y2_train)
baseline_3 = LinearRegression().fit(X_train, y3_train)
baseline_4 = LinearRegression().fit(X_train, y4_train)
baseline_5 = LinearRegression().fit(X_train, y5_train)
baseline_6 = LinearRegression().fit(X_train, y6_train)

baseline_y1_pred = baseline_1.predict(X_test)
baseline_y2_pred = baseline_2.predict(X_test)
baseline_y3_pred = baseline_3.predict(X_test)
baseline_y4_pred = baseline_4.predict(X_test)
baseline_y5_pred = baseline_5.predict(X_test)
baseline_y6_pred = baseline_6.predict(X_test)

baseline_1_r2_score = r2_score(y1_test, baseline_y1_pred)
baseline_2_r2_score = r2_score(y2_test, baseline_y2_pred)
baseline_3_r2_score = r2_score(y3_test, baseline_y3_pred)
baseline_4_r2_score = r2_score(y4_test, baseline_y4_pred)
baseline_5_r2_score = r2_score(y5_test, baseline_y5_pred)
baseline_6_r2_score = r2_score(y6_test, baseline_y6_pred)

baseline_1_rmse_score = np.sqrt(mean_squared_error(y1_test, baseline_y1_pred))
baseline_2_rmse_score = np.sqrt(mean_squared_error(y2_test, baseline_y2_pred))
baseline_3_rmse_score = np.sqrt(mean_squared_error(y3_test, baseline_y3_pred))
baseline_4_rmse_score = np.sqrt(mean_squared_error(y4_test, baseline_y4_pred))
baseline_5_rmse_score = np.sqrt(mean_squared_error(y5_test, baseline_y5_pred))
baseline_6_rmse_score = np.sqrt(mean_squared_error(y6_test, baseline_y6_pred))

In [ ]:
print('R-squared - baseline 1: ' + str(baseline_1_r2_score))
print('R-squared - baseline 2: ' + str(baseline_2_r2_score))
print('R-squared - baseline 3: ' + str(baseline_3_r2_score))
print('R-squared - baseline 4: ' + str(baseline_4_r2_score))
print('R-squared - baseline 5: ' + str(baseline_5_r2_score))
print('R-squared - baseline 6: ' + str(baseline_6_r2_score))
print()
print('RMSE - baseline 1: ' + str(baseline_1_rmse_score))
print('RMSE - baseline 2: ' + str(baseline_2_rmse_score))
print('RMSE - baseline 3: ' + str(baseline_3_rmse_score))
print('RMSE - baseline 4: ' + str(baseline_4_rmse_score))
print('RMSE - baseline 5: ' + str(baseline_5_rmse_score))
print('RMSE - baseline 6: ' + str(baseline_6_rmse_score))

R-squared - baseline 1: -0.5218806289771014
R-squared - baseline 2: -1.4047906050983205
R-squared - baseline 3: -0.5738990518978604
R-squared - baseline 4: -0.36650869719183765
R-squared - baseline 5: -0.702065498589513
R-squared - baseline 6: -5.127319385350991

RMSE - baseline 1: 0.011496012035895679
RMSE - baseline 2: 0.0030760583768864266
RMSE - baseline 3: 0.010649718410449368
RMSE - baseline 4: 0.010126184856627056
RMSE - baseline 5: 0.010776522598027841
RMSE - baseline 6: 0.017505941510283056


In [ ]:
# KNN 1
RMSE_values_1 = []
for K in range(200):
  K = K + 1
  knn_1 = KNeighborsRegressor(n_neighbors = K).fit(X_train, y1_train)
  knn_y1_pred = knn_1.predict(X_test)
  knn_1_rmse_score = np.sqrt(mean_squared_error(y1_test, knn_y1_pred))
  RMSE_values_1.append(knn_1_rmse_score)
knn_1_min_rmse = min(RMSE_values_1)
print('Optimal K for KNN 1 is: ' + str(RMSE_values_1.index(knn_1_min_rmse) + 1))

# KNN 2
RMSE_values_2 = []
for K in range(200):
  K = K + 1
  knn_2 = KNeighborsRegressor(n_neighbors = K).fit(X_train, y2_train)
  knn_y2_pred = knn_2.predict(X_test)
  knn_2_rmse_score = np.sqrt(mean_squared_error(y2_test, knn_y2_pred))
  RMSE_values_2.append(knn_2_rmse_score)
knn_2_min_rmse = min(RMSE_values_2)
print('Optimal K for KNN 2 is: ' + str(RMSE_values_2.index(knn_2_min_rmse) + 1))

# KNN 3
RMSE_values_3 = []
for K in range(200):
  K = K + 1
  knn_3 = KNeighborsRegressor(n_neighbors = K).fit(X_train, y3_train)
  knn_y3_pred = knn_3.predict(X_test)
  knn_3_rmse_score = np.sqrt(mean_squared_error(y3_test, knn_y3_pred))
  RMSE_values_3.append(knn_3_rmse_score)
knn_3_min_rmse = min(RMSE_values_3)
print('Optimal K for KNN 3 is: ' + str(RMSE_values_3.index(knn_3_min_rmse) + 1))

# KNN 4
RMSE_values_4 = []
for K in range(200):
  K = K + 1
  knn_4 = KNeighborsRegressor(n_neighbors = K).fit(X_train, y4_train)
  knn_y4_pred = knn_4.predict(X_test)
  knn_4_rmse_score = np.sqrt(mean_squared_error(y4_test, knn_y4_pred))
  RMSE_values_4.append(knn_4_rmse_score)
knn_4_min_rmse = min(RMSE_values_4)
print('Optimal K for KNN 4 is: ' + str(RMSE_values_4.index(knn_4_min_rmse) + 1))

# KNN 5
RMSE_values_5 = []
for K in range(200):
  K = K + 1
  knn_5 = KNeighborsRegressor(n_neighbors = K).fit(X_train, y5_train)
  knn_y5_pred = knn_5.predict(X_test)
  knn_5_rmse_score = np.sqrt(mean_squared_error(y5_test, knn_y5_pred))
  RMSE_values_5.append(knn_5_rmse_score)
knn_5_min_rmse = min(RMSE_values_5)
print('Optimal K for KNN 5 is: ' + str(RMSE_values_5.index(knn_5_min_rmse) + 1))

# KNN 6
RMSE_values_6 = []
for K in range(200):
  K = K + 1
  knn_6 = KNeighborsRegressor(n_neighbors = K).fit(X_train, y6_train)
  knn_y6_pred = knn_6.predict(X_test)
  knn_6_rmse_score = np.sqrt(mean_squared_error(y6_test, knn_y6_pred))
  RMSE_values_6.append(knn_6_rmse_score)
knn_6_min_rmse = min(RMSE_values_6)
print('Optimal K for KNN 6 is: ' + str(RMSE_values_6.index(knn_6_min_rmse) + 1))

Optimal K for KNN 1 is: 52
Optimal K for KNN 2 is: 58
Optimal K for KNN 3 is: 53
Optimal K for KNN 4 is: 52
Optimal K for KNN 5 is: 176
Optimal K for KNN 6 is: 51


In [ ]:
# KNN 1
knn_1 = KNeighborsRegressor(n_neighbors = 52).fit(X_train, y1_train)
knn_y1_pred = knn_1.predict(X_test)
knn_1_r2_score = r2_score(y1_test, knn_y1_pred)
knn_1_rmse_score = np.sqrt(mean_squared_error(y1_test, knn_y1_pred))
print('R-squared - KNN 1: ' + str(knn_1_r2_score))
print('RMSE - KNN 1: ' + str(knn_1_rmse_score))
print()

# KNN 2
knn_2 = KNeighborsRegressor(n_neighbors = 58).fit(X_train, y2_train)
knn_y2_pred = knn_2.predict(X_test)
knn_2_r2_score = r2_score(y2_test, knn_y2_pred)
knn_2_rmse_score = np.sqrt(mean_squared_error(y2_test, knn_y2_pred))
print('R-squared - KNN 2: ' + str(knn_2_r2_score))
print('RMSE - KNN 2: ' + str(knn_2_rmse_score))
print()

# KNN 3
knn_3 = KNeighborsRegressor(n_neighbors = 53).fit(X_train, y3_train)
knn_y3_pred = knn_3.predict(X_test)
knn_3_r2_score = r2_score(y3_test, knn_y3_pred)
knn_3_rmse_score = np.sqrt(mean_squared_error(y3_test, knn_y3_pred))
print('R-squared - KNN 3: ' + str(knn_3_r2_score))
print('RMSE - KNN 3: ' + str(knn_3_rmse_score))
print()

# KNN 4
knn_4 = KNeighborsRegressor(n_neighbors = 52).fit(X_train, y4_train)
knn_y4_pred = knn_4.predict(X_test)
knn_4_r2_score = r2_score(y4_test, knn_y4_pred)
knn_4_rmse_score = np.sqrt(mean_squared_error(y4_test, knn_y4_pred))
print('R-squared - KNN 4: ' + str(round(knn_4_r2_score, 2)))
print('RMSE - KNN 4: ' + str(round(knn_4_rmse_score, 2)))
print()

# KNN 5
knn_5 = KNeighborsRegressor(n_neighbors = 176).fit(X_train, y5_train)
knn_y5_pred = knn_5.predict(X_test)
knn_5_r2_score = r2_score(y5_test, knn_y5_pred)
knn_5_rmse_score = np.sqrt(mean_squared_error(y5_test, knn_y5_pred))
print('R-squared - KNN 5: ' + str(knn_5_r2_score))
print('RMSE - KNN 5: ' + str(knn_5_rmse_score))
print()

# KNN 6
knn_6 = KNeighborsRegressor(n_neighbors = 51).fit(X_train, y6_train)
knn_y6_pred = knn_6.predict(X_test)
knn_6_r2_score = r2_score(y6_test, knn_y6_pred)
knn_6_rmse_score = np.sqrt(mean_squared_error(y6_test, knn_y6_pred))
print('R-squared - KNN 6: ' + str(knn_6_r2_score))
print('RMSE - KNN 6: ' + str(knn_6_rmse_score))

R-squared - KNN 1: 0.0336387496193612
RMSE - KNN 1: 0.009160657980175431

R-squared - KNN 2: 0.06245194662948872
RMSE - KNN 2: 0.0019206698016760572

R-squared - KNN 3: 0.03212033983186957
RMSE - KNN 3: 0.00835142085079818

R-squared - KNN 4: 0.02
RMSE - KNN 4: 0.01

R-squared - KNN 5: -0.025241949614604353
RMSE - KNN 5: 0.008363801048339408

R-squared - KNN 6: 0.04046359634039487
RMSE - KNN 6: 0.006927570303833208


Multilayer Perceptron

In [ ]:
def mlp_builder(hp):
  hp_units = hp.Int('units', min_value = 50, max_value = 500, step = 50)
  hp_learning_rate = hp.Choice('learning_rate',
                               values = [1e-1, 1e-2, 1e-3, 1e-4])
  hp_activation = hp.Choice('activation',
                            values = ['relu', 'tanh', 'sigmoid'])
  model = tf.keras.Sequential()
  model.add((keras.layers.Dense(units = hp_units,
                                     activation = hp_activation,
                                     input_dim = X_train.shape[1])))
  model.add(keras.layers.Dense(units = hp_units,
                               activation = hp_activation))
  model.add(keras.layers.Dense(units = hp_units,
                               activation = hp_activation))
  model.add(keras.layers.Dense(units = hp_units,
                               activation = hp_activation))
  model.add(keras.layers.Dense(y1_train.shape[1]))

  model.compile(optimizer = Adam(learning_rate = hp_learning_rate),
                loss = 'mse',
                metrics = tf.keras.metrics.RootMeanSquaredError())
  return model

In [ ]:
tuner = kt.Hyperband(mlp_builder,
                     objective = 'val_loss',
                     max_epochs = 100,
                     directory = 'keras_tuner_dir',
                     project_name = 'keras_mlp',
                     overwrite = True)

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 5)

In [ ]:
tuner.search(X_train, y1_train,
             epochs = 100, validation_data = (X_val, y1_val),
             callbacks = [stop_early])

Trial 151 Complete [00h 00m 11s]
val_loss: 9.598083852324635e-05

Best val_loss So Far: 9.386929013999179e-05
Total elapsed time: 00h 09m 43s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hps_mlp_1=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"""
The hyperparameter search for NAESX MLP is complete.
The optimal number of units in the first hidden
layer is {best_hps_mlp_1.get('units')},
the optimal activation function is {best_hps_mlp_1.get('activation')}
and the optimal learning rate for the optimizer
is {best_hps_mlp_1.get('learning_rate')}.
""")


The hyperparameter search for NAESX MLP is complete.
The optimal number of units in the first hidden
layer is 100,
the optimal activation function is sigmoid
and the optimal learning rate for the optimizer
is 0.0001.



In [ ]:
model_mlp_1 = tuner.hypermodel.build(best_hps_mlp_1)

history = model_mlp_1.fit(X_train, y1_train,
                    epochs = 100, validation_data = (X_val, y1_val))

val_rmse_per_epoch = history.history['val_root_mean_squared_error']

best_epoch = val_rmse_per_epoch.index(min(val_rmse_per_epoch))

print('Best epoch: %d' % (best_epoch,))

Epoch 1/100
136/136 [==============================] - 1s 5ms/step - loss: 3.6430e-04 - root_mean_squared_error: 0.0191 - val_loss: 9.4969e-05 - val_root_mean_squared_error: 0.0097
Epoch 2/100
136/136 [==============================] - 0s 3ms/step - loss: 2.1214e-04 - root_mean_squared_error: 0.0146 - val_loss: 9.7596e-05 - val_root_mean_squared_error: 0.0099
Epoch 3/100
136/136 [==============================] - 0s 4ms/step - loss: 2.1897e-04 - root_mean_squared_error: 0.0148 - val_loss: 9.4207e-05 - val_root_mean_squared_error: 0.0097
Epoch 4/100
136/136 [==============================] - 0s 3ms/step - loss: 2.1581e-04 - root_mean_squared_error: 0.0147 - val_loss: 9.8554e-05 - val_root_mean_squared_error: 0.0099
Epoch 5/100
136/136 [==============================] - 0s 3ms/step - loss: 2.1701e-04 - root_mean_squared_error: 0.0147 - val_loss: 9.4730e-05 - val_root_mean_squared_error: 0.0097
Epoch 6/100
136/136 [==============================] - 0s 3ms/step - loss: 2.1684e-04 - root_me

In [ ]:
hypermodel_mlp_1 = tuner.hypermodel.build(best_hps_mlp_1)
hypermodel_mlp_1.fit(X_train, y1_train,
               epochs=best_epoch,
               validation_data = (X_val, y1_val),
               shuffle = False)

Epoch 1/73
136/136 [==============================] - 1s 4ms/step - loss: 0.0019 - root_mean_squared_error: 0.0433 - val_loss: 9.4390e-05 - val_root_mean_squared_error: 0.0097
Epoch 2/73
136/136 [==============================] - 0s 3ms/step - loss: 2.1156e-04 - root_mean_squared_error: 0.0145 - val_loss: 9.3985e-05 - val_root_mean_squared_error: 0.0097
Epoch 3/73
136/136 [==============================] - 0s 3ms/step - loss: 2.1316e-04 - root_mean_squared_error: 0.0146 - val_loss: 9.4351e-05 - val_root_mean_squared_error: 0.0097
Epoch 4/73
136/136 [==============================] - 0s 3ms/step - loss: 2.1359e-04 - root_mean_squared_error: 0.0146 - val_loss: 9.4166e-05 - val_root_mean_squared_error: 0.0097
Epoch 5/73
136/136 [==============================] - 0s 3ms/step - loss: 2.1380e-04 - root_mean_squared_error: 0.0146 - val_loss: 9.4352e-05 - val_root_mean_squared_error: 0.0097
Epoch 6/73
136/136 [==============================] - 0s 3ms/step - loss: 2.1388e-04 - root_mean_squared

In [ ]:
y1_pred = hypermodel_mlp_1.predict(X_test)

In [ ]:
### Evaluate hypermodel using test sets.
eval_result = hypermodel_mlp_1.evaluate(X_test, y1_test)
print("[test loss, test rmse]:", eval_result)

30/30 [==============================] - 0s 1ms/step - loss: 8.7685e-05 - root_mean_squared_error: 0.0094
[test loss, test rmse]: [8.768451516516507e-05, 0.009364001452922821]


In [ ]:
mlp_1_r2_score = r2_score(y1_test, y1_pred)
RMSE_mlp_1 = np.sqrt(mean_squared_error(y1_test, y1_pred))
print('NAESX - R-squared - MLP: ' + str(mlp_1_r2_score))
print('NAESX - RMSE - MLP: ' + str(RMSE_mlp_1))

NAESX - R-squared - MLP: -0.009739038477000417
NAESX - RMSE - MLP: 0.00936400180702326


**VBMFX**

In [ ]:
def mlp_builder(hp):
  hp_units = hp.Int('units', min_value = 50, max_value = 500, step = 50)
  hp_learning_rate = hp.Choice('learning_rate',
                               values = [1e-1, 1e-2, 1e-3, 1e-4])
  hp_activation = hp.Choice('activation',
                            values = ['relu', 'tanh', 'sigmoid'])
  model = tf.keras.Sequential()
  model.add((keras.layers.Dense(units = hp_units,
                                activation = hp_activation,
                                input_dim = X_train.shape[1])))
  model.add(keras.layers.Dense(units = hp_units, activation = hp_activation))
  model.add(keras.layers.Dense(units = hp_units, activation = hp_activation))
  model.add(keras.layers.Dense(units = hp_units, activation = hp_activation))
  model.add(keras.layers.Dense(y2_train.shape[1]))

  model.compile(optimizer = Adam(learning_rate = hp_learning_rate),
                loss = 'mse',
                metrics = tf.keras.metrics.RootMeanSquaredError())
  return model

In [ ]:
tuner = kt.Hyperband(mlp_builder,
                     objective = 'val_loss',
                     max_epochs = 100,
                     directory = 'keras_tuner_dir',
                     project_name = 'keras_mlp',
                     overwrite = True)

In [ ]:
tuner.search(X_train, y2_train,
             epochs = 100, validation_data = (X_val, y2_val),
             callbacks = [stop_early])

Trial 152 Complete [00h 00m 05s]
val_loss: 5.388070348999463e-05

Best val_loss So Far: 5.791890089312801e-06
Total elapsed time: 00h 11m 18s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hps_mlp_2=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"""
The hyperparameter search for VBMFX MLP is complete.
The optimal number of units in the first hidden
layer is {best_hps_mlp_2.get('units')},
the optimal activation function is {best_hps_mlp_2.get('activation')}
and the optimal learning rate for the optimizer
is {best_hps_mlp_2.get('learning_rate')}.
""")


The hyperparameter search for VBMFX MLP is complete.
The optimal number of units in the first hidden
layer is 450,
the optimal activation function is sigmoid
and the optimal learning rate for the optimizer
is 0.0001.



In [ ]:
model = tuner.hypermodel.build(best_hps_mlp_2)

history = model.fit(X_train, y2_train,
                    epochs=100, validation_data = (X_val, y2_val))

val_rmse_per_epoch = history.history['val_root_mean_squared_error']

best_epoch = val_rmse_per_epoch.index(min(val_rmse_per_epoch))

print('Best epoch: %d' % (best_epoch,))

Epoch 1/100
136/136 [==============================] - 2s 13ms/step - loss: 0.0252 - root_mean_squared_error: 0.1588 - val_loss: 8.4779e-06 - val_root_mean_squared_error: 0.0029
Epoch 2/100
136/136 [==============================] - 2s 12ms/step - loss: 7.7921e-06 - root_mean_squared_error: 0.0028 - val_loss: 8.0223e-06 - val_root_mean_squared_error: 0.0028
Epoch 3/100
136/136 [==============================] - 2s 12ms/step - loss: 7.6776e-06 - root_mean_squared_error: 0.0028 - val_loss: 7.4670e-06 - val_root_mean_squared_error: 0.0027
Epoch 4/100
136/136 [==============================] - 2s 13ms/step - loss: 7.7674e-06 - root_mean_squared_error: 0.0028 - val_loss: 8.0941e-06 - val_root_mean_squared_error: 0.0028
Epoch 5/100
136/136 [==============================] - 2s 12ms/step - loss: 7.6575e-06 - root_mean_squared_error: 0.0028 - val_loss: 6.7991e-06 - val_root_mean_squared_error: 0.0026
Epoch 6/100
136/136 [==============================] - 2s 12ms/step - loss: 7.8145e-06 - root_

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps_mlp_2)
hypermodel.fit(X_train, y2_train,
               epochs=best_epoch,
               validation_data = (X_val, y2_val),
               shuffle = False)

Epoch 1/24
136/136 [==============================] - 2s 13ms/step - loss: 9.8912e-04 - root_mean_squared_error: 0.0315 - val_loss: 5.8829e-06 - val_root_mean_squared_error: 0.0024
Epoch 2/24
136/136 [==============================] - 2s 12ms/step - loss: 9.1520e-06 - root_mean_squared_error: 0.0030 - val_loss: 6.7576e-06 - val_root_mean_squared_error: 0.0026
Epoch 3/24
136/136 [==============================] - 2s 12ms/step - loss: 1.0284e-05 - root_mean_squared_error: 0.0032 - val_loss: 6.9699e-06 - val_root_mean_squared_error: 0.0026
Epoch 4/24
136/136 [==============================] - 2s 12ms/step - loss: 1.1197e-05 - root_mean_squared_error: 0.0033 - val_loss: 1.8430e-05 - val_root_mean_squared_error: 0.0043
Epoch 5/24
136/136 [==============================] - 2s 12ms/step - loss: 1.5478e-05 - root_mean_squared_error: 0.0039 - val_loss: 6.1542e-06 - val_root_mean_squared_error: 0.0025
Epoch 6/24
136/136 [==============================] - 2s 12ms/step - loss: 1.3791e-05 - root_me

In [ ]:
y2_pred = hypermodel.predict(X_test)

In [ ]:
eval_result = hypermodel.evaluate(X_test, y2_test)
print("[test loss, test rmse]:", eval_result)

30/30 [==============================] - 0s 4ms/step - loss: 4.1947e-06 - root_mean_squared_error: 0.0020
[test loss, test rmse]: [4.1947400859498885e-06, 0.002048106398433447]


In [ ]:
mlp_2_r2_score = r2_score(y2_test, y2_pred)
RMSE_mlp_2 = np.sqrt(mean_squared_error(y2_test, y2_pred))
print('VBMFX - R-squared - MLP: ' + str(mlp_2_r2_score))
print('VBMFX - RMSE - MLP: ' + str(RMSE_mlp_2))

VBMFX - R-squared - MLP: -0.06609261129688648
VBMFX - RMSE - MLP: 0.0020481106062653813


**VEIEX**

In [ ]:
def mlp_builder(hp):
  hp_units = hp.Int('units', min_value = 50, max_value = 500, step = 50)
  hp_learning_rate = hp.Choice('learning_rate',
                               values = [1e-1, 1e-2, 1e-3, 1e-4])
  hp_activation = hp.Choice('activation',
                            values = ['relu', 'tanh', 'sigmoid'])
  model = tf.keras.Sequential()
  model.add((keras.layers.Dense(units = hp_units,
                                     activation = hp_activation,
                                     input_dim = X_train.shape[1])))
  model.add(keras.layers.Dense(units = hp_units, activation = hp_activation))
  model.add(keras.layers.Dense(units = hp_units, activation = hp_activation))
  model.add(keras.layers.Dense(units = hp_units, activation = hp_activation))
  model.add(keras.layers.Dense(y3_train.shape[1]))

  model.compile(optimizer = Adam(learning_rate = hp_learning_rate),
                loss = 'mse',
                metrics = tf.keras.metrics.RootMeanSquaredError())
  return model

In [ ]:
tuner = kt.Hyperband(mlp_builder,
                     objective = 'val_loss',
                     max_epochs = 100,
                     directory = 'keras_tuner_dir',
                     project_name = 'keras_mlp',
                     overwrite = True)

In [ ]:
tuner.search(X_train, y3_train,
             epochs = 100, validation_data = (X_val, y3_val),
             callbacks = [stop_early])

Trial 163 Complete [00h 00m 05s]
val_loss: 0.00017557542014401406

Best val_loss So Far: 0.00010575516353128478
Total elapsed time: 00h 11m 51s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hps_mlp_3=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"""
The hyperparameter search for VEIEX MLP is complete.
The optimal number of units in the first hidden
layer is {best_hps_mlp_3.get('units')},
the optimal activation function is {best_hps_mlp_3.get('activation')}
and the optimal learning rate for the optimizer
is {best_hps_mlp_3.get('learning_rate')}.
""")


The hyperparameter search for VEIEX MLP is complete.
The optimal number of units in the first hidden
layer is 200,
the optimal activation function is tanh
and the optimal learning rate for the optimizer
is 0.001.



In [ ]:
model = tuner.hypermodel.build(best_hps_mlp_3)

history = model.fit(X_train, y3_train,
                    epochs=100, validation_data = (X_val, y3_val))

val_rmse_per_epoch = history.history['val_root_mean_squared_error']

best_epoch = val_rmse_per_epoch.index(min(val_rmse_per_epoch))

print('Best epoch: %d' % (best_epoch,))

Epoch 1/100
136/136 [==============================] - 1s 5ms/step - loss: 0.0089 - root_mean_squared_error: 0.0945 - val_loss: 2.0062e-04 - val_root_mean_squared_error: 0.0142
Epoch 2/100
136/136 [==============================] - 1s 4ms/step - loss: 0.0011 - root_mean_squared_error: 0.0338 - val_loss: 2.6648e-04 - val_root_mean_squared_error: 0.0163
Epoch 3/100
136/136 [==============================] - 1s 4ms/step - loss: 9.7643e-04 - root_mean_squared_error: 0.0312 - val_loss: 7.6028e-04 - val_root_mean_squared_error: 0.0276
Epoch 4/100
136/136 [==============================] - 1s 4ms/step - loss: 8.2597e-04 - root_mean_squared_error: 0.0287 - val_loss: 0.0011 - val_root_mean_squared_error: 0.0337
Epoch 5/100
136/136 [==============================] - 1s 4ms/step - loss: 6.2350e-04 - root_mean_squared_error: 0.0250 - val_loss: 0.0025 - val_root_mean_squared_error: 0.0504
Epoch 6/100
136/136 [==============================] - 1s 4ms/step - loss: 0.0024 - root_mean_squared_error: 0.

In [ ]:
### Pass the optimal hyperparameters to hypermodel.
hypermodel = tuner.hypermodel.build(best_hps_mlp_3)

### Retrain the model with the optimal number of epochs.
hypermodel.fit(X_train, y3_train,
               epochs=best_epoch,
               validation_data = (X_val, y3_val),
               shuffle = False)

Epoch 1/89
136/136 [==============================] - 1s 5ms/step - loss: 0.0646 - root_mean_squared_error: 0.2542 - val_loss: 0.3257 - val_root_mean_squared_error: 0.5707
Epoch 2/89
136/136 [==============================] - 1s 4ms/step - loss: 0.0683 - root_mean_squared_error: 0.2614 - val_loss: 0.0363 - val_root_mean_squared_error: 0.1906
Epoch 3/89
136/136 [==============================] - 1s 4ms/step - loss: 0.0294 - root_mean_squared_error: 0.1714 - val_loss: 0.0150 - val_root_mean_squared_error: 0.1223
Epoch 4/89
136/136 [==============================] - 1s 4ms/step - loss: 0.0016 - root_mean_squared_error: 0.0403 - val_loss: 7.4566e-04 - val_root_mean_squared_error: 0.0273
Epoch 5/89
136/136 [==============================] - 1s 4ms/step - loss: 2.6493e-04 - root_mean_squared_error: 0.0163 - val_loss: 3.0622e-04 - val_root_mean_squared_error: 0.0175
Epoch 6/89
136/136 [==============================] - 1s 4ms/step - loss: 2.2097e-04 - root_mean_squared_error: 0.0149 - val_los

In [ ]:
y3_pred = hypermodel.predict(X_test)

In [ ]:
eval_result = hypermodel.evaluate(X_test, y3_test)
print("[test loss, test rmse]:", eval_result)

30/30 [==============================] - 0s 1ms/step - loss: 4.1981e-04 - root_mean_squared_error: 0.0205
[test loss, test rmse]: [0.00041980540845543146, 0.020489152520895004]


In [ ]:
mlp_3_r2_score = r2_score(y3_test, y3_pred)
RMSE_mlp_3 = np.sqrt(mean_squared_error(y3_test, y3_pred))
print('VEIEX - R-squared - MLP: ' + str(mlp_3_r2_score))
print('VEIEX - RMSE - MLP: ' + str(RMSE_mlp_3))

VEIEX - R-squared - MLP: -4.825707819840065
VEIEX - RMSE - MLP: 0.020489154512588998


**VFINX**

In [ ]:
def mlp_builder(hp):
  hp_units = hp.Int('units', min_value = 50, max_value = 500, step = 50)
  hp_learning_rate = hp.Choice('learning_rate',
                               values = [1e-1, 1e-2, 1e-3, 1e-4])
  hp_activation = hp.Choice('activation',
                            values = ['relu', 'tanh', 'sigmoid'])
  model = tf.keras.Sequential()
  model.add((keras.layers.Dense(units = hp_units,
                                     activation = hp_activation,
                                     input_dim = X_train.shape[1])))
  model.add(keras.layers.Dense(units = hp_units, activation = hp_activation))
  model.add(keras.layers.Dense(units = hp_units, activation = hp_activation))
  model.add(keras.layers.Dense(units = hp_units, activation = hp_activation))
  model.add(keras.layers.Dense(y4_train.shape[1]))

  model.compile(optimizer = Adam(learning_rate = hp_learning_rate),
                loss = 'mse',
                metrics = tf.keras.metrics.RootMeanSquaredError())
  return model

In [ ]:
tuner = kt.Hyperband(mlp_builder,
                     objective = 'val_loss',
                     max_epochs = 100,
                     directory = 'keras_tuner_dir',
                     project_name = 'keras_mlp',
                     overwrite = True)

In [ ]:
tuner.search(X_train, y4_train,
             epochs = 100, validation_data = (X_val, y4_val),
             callbacks = [stop_early])

Trial 154 Complete [00h 00m 02s]
val_loss: 7.55689907236956e-05

Best val_loss So Far: 7.340312004089355e-05
Total elapsed time: 00h 10m 57s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hps_mlp_4=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"""
The hyperparameter search for VFINX MLP is complete.
The optimal number of units in the first hidden
layer is {best_hps_mlp_4.get('units')},
the optimal activation function is {best_hps_mlp_4.get('activation')}
and the optimal learning rate for the optimizer
is {best_hps_mlp_4.get('learning_rate')}.
""")


The hyperparameter search for VFINX MLP is complete.
The optimal number of units in the first hidden
layer is 100,
the optimal activation function is relu
and the optimal learning rate for the optimizer
is 0.01.



In [ ]:
model = tuner.hypermodel.build(best_hps_mlp_4)

history = model.fit(X_train, y4_train,
                    epochs=100, validation_data = (X_val, y4_val))

val_rmse_per_epoch = history.history['val_root_mean_squared_error']

best_epoch = val_rmse_per_epoch.index(min(val_rmse_per_epoch))

print('Best epoch: %d' % (best_epoch,))

Epoch 1/100
136/136 [==============================] - 1s 5ms/step - loss: 0.1059 - root_mean_squared_error: 0.3254 - val_loss: 0.0351 - val_root_mean_squared_error: 0.1874
Epoch 2/100
136/136 [==============================] - 0s 3ms/step - loss: 2.7845e-04 - root_mean_squared_error: 0.0167 - val_loss: 1.1841e-04 - val_root_mean_squared_error: 0.0109
Epoch 3/100
136/136 [==============================] - 0s 4ms/step - loss: 2.1709e-04 - root_mean_squared_error: 0.0147 - val_loss: 1.4588e-04 - val_root_mean_squared_error: 0.0121
Epoch 4/100
136/136 [==============================] - 0s 4ms/step - loss: 0.0035 - root_mean_squared_error: 0.0591 - val_loss: 1.1807e-04 - val_root_mean_squared_error: 0.0109
Epoch 5/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0045 - root_mean_squared_error: 0.0671 - val_loss: 1.0109e-04 - val_root_mean_squared_error: 0.0101
Epoch 6/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0233 - root_mean_squared_error: 0.

In [ ]:
### Pass the optimal hyperparameters to hypermodel.
hypermodel = tuner.hypermodel.build(best_hps_mlp_4)

### Retrain the model with the optimal number of epochs.
hypermodel.fit(X_train, y4_train,
               epochs=best_epoch,
               validation_data = (X_val, y4_val),
               shuffle = False)

Epoch 1/37
136/136 [==============================] - 1s 4ms/step - loss: 0.2979 - root_mean_squared_error: 0.5458 - val_loss: 0.1027 - val_root_mean_squared_error: 0.3205
Epoch 2/37
136/136 [==============================] - 0s 3ms/step - loss: 0.0080 - root_mean_squared_error: 0.0895 - val_loss: 1.8065 - val_root_mean_squared_error: 1.3441
Epoch 3/37
136/136 [==============================] - 0s 4ms/step - loss: 0.0348 - root_mean_squared_error: 0.1865 - val_loss: 0.4081 - val_root_mean_squared_error: 0.6388
Epoch 4/37
136/136 [==============================] - 0s 4ms/step - loss: 0.0047 - root_mean_squared_error: 0.0686 - val_loss: 0.1478 - val_root_mean_squared_error: 0.3844
Epoch 5/37
136/136 [==============================] - 0s 3ms/step - loss: 0.0102 - root_mean_squared_error: 0.1010 - val_loss: 0.2315 - val_root_mean_squared_error: 0.4811
Epoch 6/37
136/136 [==============================] - 0s 3ms/step - loss: 0.0045 - root_mean_squared_error: 0.0669 - val_loss: 1.2248e-04 - 

In [ ]:
y4_pred = hypermodel.predict(X_test)

In [ ]:
eval_result = hypermodel.evaluate(X_test, y4_test)
print("[test loss, test rmse]:", eval_result)

30/30 [==============================] - 0s 1ms/step - loss: 9.0578e-05 - root_mean_squared_error: 0.0095
[test loss, test rmse]: [9.057806892087683e-05, 0.009517251513898373]


In [ ]:
mlp_4_r2_score = r2_score(y4_test, y4_pred)
RMSE_mlp_4 = np.sqrt(mean_squared_error(y4_test, y4_pred))
print('VFINX - R-squared - MLP: ' + str(mlp_4_r2_score))
print('VFINX - RMSE - MLP: ' + str(RMSE_mlp_4))

VFINX - R-squared - MLP: -0.2071014542658205
VFINX - RMSE - MLP: 0.009517251320680856


**VGSLX**

In [ ]:
def mlp_builder(hp):
  hp_units = hp.Int('units', min_value = 50, max_value = 500, step = 50)
  hp_learning_rate = hp.Choice('learning_rate',
                               values = [1e-1, 1e-2, 1e-3, 1e-4])
  hp_activation = hp.Choice('activation',
                            values = ['relu', 'tanh', 'sigmoid'])
  model = tf.keras.Sequential()
  model.add((keras.layers.Dense(units = hp_units,
                                     activation = hp_activation,
                                     input_dim = X_train.shape[1])))
  model.add(keras.layers.Dense(units = hp_units, activation = hp_activation))
  model.add(keras.layers.Dense(units = hp_units, activation = hp_activation))
  model.add(keras.layers.Dense(units = hp_units, activation = hp_activation))  
  model.add(keras.layers.Dense(y5_train.shape[1]))

  model.compile(optimizer = Adam(learning_rate = hp_learning_rate),
                loss = 'mse',
                metrics = tf.keras.metrics.RootMeanSquaredError())
  return model

In [ ]:
tuner = kt.Hyperband(mlp_builder,
                     objective = 'val_loss',
                     max_epochs = 100,
                     directory = 'keras_tuner_dir',
                     project_name = 'keras_mlp',
                     overwrite = True)

In [ ]:
tuner.search(X_train, y5_train,
             epochs = 100, validation_data = (X_val, y5_val),
             callbacks = [stop_early])

Trial 154 Complete [00h 00m 03s]
val_loss: 0.04570956155657768

Best val_loss So Far: 0.00010552429012022913
Total elapsed time: 00h 11m 10s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hps_mlp_5=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"""
The hyperparameter search for VGSLX MLP is complete.
The optimal number of units in the first hidden
layer is {best_hps_mlp_5.get('units')},
the optimal activation function is {best_hps_mlp_5.get('activation')}
and the optimal learning rate for the optimizer
is {best_hps_mlp_5.get('learning_rate')}.
""")


The hyperparameter search for VGSLX MLP is complete.
The optimal number of units in the first hidden
layer is 50,
the optimal activation function is sigmoid
and the optimal learning rate for the optimizer
is 0.0001.



In [ ]:
model = tuner.hypermodel.build(best_hps_mlp_5)

history = model.fit(X_train, y5_train,
                    epochs=100, validation_data = (X_val, y5_val))

val_rmse_per_epoch = history.history['val_root_mean_squared_error']

best_epoch = val_rmse_per_epoch.index(min(val_rmse_per_epoch))

print('Best epoch: %d' % (best_epoch,))

Epoch 1/100
136/136 [==============================] - 1s 5ms/step - loss: 0.0965 - root_mean_squared_error: 0.3107 - val_loss: 1.7268e-04 - val_root_mean_squared_error: 0.0131
Epoch 2/100
136/136 [==============================] - 0s 3ms/step - loss: 4.4254e-04 - root_mean_squared_error: 0.0210 - val_loss: 1.0817e-04 - val_root_mean_squared_error: 0.0104
Epoch 3/100
136/136 [==============================] - 0s 3ms/step - loss: 4.3853e-04 - root_mean_squared_error: 0.0209 - val_loss: 1.0723e-04 - val_root_mean_squared_error: 0.0104
Epoch 4/100
136/136 [==============================] - 0s 3ms/step - loss: 4.3846e-04 - root_mean_squared_error: 0.0209 - val_loss: 1.0746e-04 - val_root_mean_squared_error: 0.0104
Epoch 5/100
136/136 [==============================] - 0s 3ms/step - loss: 4.3849e-04 - root_mean_squared_error: 0.0209 - val_loss: 1.0918e-04 - val_root_mean_squared_error: 0.0104
Epoch 6/100
136/136 [==============================] - 1s 4ms/step - loss: 4.3959e-04 - root_mean_s

In [ ]:
### Pass the optimal hyperparameters to hypermodel.
hypermodel = tuner.hypermodel.build(best_hps_mlp_5)

### Retrain the model with the optimal number of epochs.
hypermodel.fit(X_train, y5_train,
               epochs=best_epoch,
               validation_data = (X_val, y5_val),
               shuffle = False)

Epoch 1/24
136/136 [==============================] - 1s 4ms/step - loss: 0.0380 - root_mean_squared_error: 0.1949 - val_loss: 1.0827e-04 - val_root_mean_squared_error: 0.0104
Epoch 2/24
136/136 [==============================] - 0s 3ms/step - loss: 4.3916e-04 - root_mean_squared_error: 0.0210 - val_loss: 1.0563e-04 - val_root_mean_squared_error: 0.0103
Epoch 3/24
136/136 [==============================] - 0s 4ms/step - loss: 4.3925e-04 - root_mean_squared_error: 0.0210 - val_loss: 1.0560e-04 - val_root_mean_squared_error: 0.0103
Epoch 4/24
136/136 [==============================] - 0s 3ms/step - loss: 4.3935e-04 - root_mean_squared_error: 0.0210 - val_loss: 1.0560e-04 - val_root_mean_squared_error: 0.0103
Epoch 5/24
136/136 [==============================] - 0s 3ms/step - loss: 4.3943e-04 - root_mean_squared_error: 0.0210 - val_loss: 1.0562e-04 - val_root_mean_squared_error: 0.0103
Epoch 6/24
136/136 [==============================] - 0s 3ms/step - loss: 4.3950e-04 - root_mean_squared

In [ ]:
y5_pred = hypermodel.predict(X_test)

In [ ]:
eval_result = hypermodel.evaluate(X_test, y5_test)
print("[test loss, test rmse]:", eval_result)

30/30 [==============================] - 0s 1ms/step - loss: 6.8929e-05 - root_mean_squared_error: 0.0083
[test loss, test rmse]: [6.892879900988191e-05, 0.008302336558699608]


In [ ]:
mlp_5_r2_score = r2_score(y5_test, y5_pred)
RMSE_mlp_5 = np.sqrt(mean_squared_error(y5_test, y5_pred))
print('VGSLX - R-squared - MLP: ' + str(mlp_5_r2_score))
print('VGSLX - RMSE - MLP: ' + str(RMSE_mlp_5))

VGSLX - R-squared - MLP: -0.010228308681075271
VGSLX - RMSE - MLP: 0.008302335450514696


**VGTSX**

In [ ]:
def mlp_builder(hp):
  hp_units = hp.Int('units', min_value = 50, max_value = 500, step = 50)
  hp_learning_rate = hp.Choice('learning_rate',
                               values = [1e-1, 1e-2, 1e-3, 1e-4])
  hp_activation = hp.Choice('activation',
                            values = ['relu', 'tanh', 'sigmoid'])
  model = tf.keras.Sequential()
  model.add((keras.layers.Dense(units = hp_units,
                                     activation = hp_activation,
                                     input_dim = X_train.shape[1])))
  model.add(keras.layers.Dense(units = hp_units, activation = hp_activation))
  model.add(keras.layers.Dense(units = hp_units, activation = hp_activation))
  model.add(keras.layers.Dense(units = hp_units, activation = hp_activation))
  model.add(keras.layers.Dense(y6_train.shape[1]))

  model.compile(optimizer = Adam(learning_rate = hp_learning_rate),
                loss = 'mse',
                metrics = tf.keras.metrics.RootMeanSquaredError())
  return model

In [ ]:
tuner = kt.Hyperband(mlp_builder,
                     objective = 'val_loss',
                     max_epochs = 100,
                     directory = 'keras_tuner_dir',
                     project_name = 'keras_mlp',
                     overwrite = True)

In [ ]:
tuner.search(X_train, y6_train,
             epochs = 100, validation_data = (X_val, y6_val),
             callbacks = [stop_early])

Trial 148 Complete [00h 00m 04s]
val_loss: 0.00010072103759739548

Best val_loss So Far: 0.00010020555782830343
Total elapsed time: 00h 10m 06s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hps_mlp_6=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"""
The hyperparameter search for VGTSX MLP is complete.
The optimal number of units in the first hidden
layer is {best_hps_mlp_6.get('units')},
the optimal activation function is {best_hps_mlp_6.get('activation')}
and the optimal learning rate for the optimizer
is {best_hps_mlp_6.get('learning_rate')}.
""")


The hyperparameter search for VGTSX MLP is complete.
The optimal number of units in the first hidden
layer is 450,
the optimal activation function is sigmoid
and the optimal learning rate for the optimizer
is 0.001.



In [ ]:
model = tuner.hypermodel.build(best_hps_mlp_6)

history = model.fit(X_train, y6_train,
                    epochs=100, validation_data = (X_val, y6_val))

val_rmse_per_epoch = history.history['val_root_mean_squared_error']

best_epoch = val_rmse_per_epoch.index(min(val_rmse_per_epoch))

print('Best epoch: %d' % (best_epoch,))

Epoch 1/100
136/136 [==============================] - 3s 15ms/step - loss: 0.0672 - root_mean_squared_error: 0.2593 - val_loss: 1.3778e-04 - val_root_mean_squared_error: 0.0117
Epoch 2/100
136/136 [==============================] - 2s 13ms/step - loss: 1.6981e-04 - root_mean_squared_error: 0.0130 - val_loss: 2.1273e-04 - val_root_mean_squared_error: 0.0146
Epoch 3/100
136/136 [==============================] - 2s 14ms/step - loss: 1.7773e-04 - root_mean_squared_error: 0.0133 - val_loss: 1.2622e-04 - val_root_mean_squared_error: 0.0112
Epoch 4/100
136/136 [==============================] - 2s 12ms/step - loss: 1.8614e-04 - root_mean_squared_error: 0.0136 - val_loss: 1.2068e-04 - val_root_mean_squared_error: 0.0110
Epoch 5/100
136/136 [==============================] - 2s 12ms/step - loss: 1.8099e-04 - root_mean_squared_error: 0.0135 - val_loss: 1.2014e-04 - val_root_mean_squared_error: 0.0110
Epoch 6/100
136/136 [==============================] - 2s 12ms/step - loss: 1.8207e-04 - root_

In [ ]:
### Pass the optimal hyperparameters to hypermodel.
hypermodel_mlp_6= tuner.hypermodel.build(best_hps_mlp_6)

### Retrain the model with the optimal number of epochs.
hypermodel_mlp_6.fit(X_train, y6_train,
               epochs=best_epoch,
               validation_data = (X_val, y6_val),
               shuffle = False)

Epoch 1/36
136/136 [==============================] - 2s 14ms/step - loss: 0.0879 - root_mean_squared_error: 0.2964 - val_loss: 1.0141e-04 - val_root_mean_squared_error: 0.0101
Epoch 2/36
136/136 [==============================] - 2s 12ms/step - loss: 1.6927e-04 - root_mean_squared_error: 0.0130 - val_loss: 1.0101e-04 - val_root_mean_squared_error: 0.0101
Epoch 3/36
136/136 [==============================] - 2s 12ms/step - loss: 1.7335e-04 - root_mean_squared_error: 0.0132 - val_loss: 1.0124e-04 - val_root_mean_squared_error: 0.0101
Epoch 4/36
136/136 [==============================] - 2s 12ms/step - loss: 1.7394e-04 - root_mean_squared_error: 0.0132 - val_loss: 1.0202e-04 - val_root_mean_squared_error: 0.0101
Epoch 5/36
136/136 [==============================] - 2s 12ms/step - loss: 1.7688e-04 - root_mean_squared_error: 0.0133 - val_loss: 1.0035e-04 - val_root_mean_squared_error: 0.0100
Epoch 6/36
136/136 [==============================] - 2s 12ms/step - loss: 1.8253e-04 - root_mean_s

In [ ]:
y6_pred = hypermodel.predict(X_test)

In [ ]:
eval_result = hypermodel.evaluate(X_test, y6_test)
print("[test loss, test rmse]:", eval_result)

30/30 [==============================] - 0s 1ms/step - loss: 5.0873e-05 - root_mean_squared_error: 0.0071
[test loss, test rmse]: [5.0872895371867344e-05, 0.0071325236931443214]


In [ ]:
mlp_6_r2_score = r2_score(y6_test, y6_pred)
RMSE_mlp_6 = np.sqrt(mean_squared_error(y6_test, y6_pred))
print('VGTSX - R-squared - MLP: ' + str(mlp_6_r2_score))
print('VGTSX - RMSE - MLP: ' + str(RMSE_mlp_6))

VGTSX - R-squared - MLP: -0.017151936012955504
VGTSX - RMSE - MLP: 0.0071325221479577195


**LSTM**

In [ ]:
X_train_2 = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_val_2 = X_val.reshape(X_val.shape[0], 1, X_val.shape[1])
X_test_2 = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

y1_train_2 = y1_train.reshape(y1_train.shape[0], 1, y1_train.shape[1])
y1_val_2 = y1_val.reshape(y1_val.shape[0], 1, y1_val.shape[1])
y1_test_2 = y1_test.reshape(y1_test.shape[0], 1, y1_test.shape[1])

y2_train_2 = y2_train.reshape(y2_train.shape[0], 1, y2_train.shape[1])
y2_val_2 = y2_val.reshape(y2_val.shape[0], 1, y2_val.shape[1])
y2_test_2 = y2_test.reshape(y2_test.shape[0], 1, y2_test.shape[1])

y3_train_2 = y3_train.reshape(y3_train.shape[0], 1, y3_train.shape[1])
y3_val_2 = y3_val.reshape(y3_val.shape[0], 1, y3_val.shape[1])
y3_test_2 = y3_test.reshape(y3_test.shape[0], 1, y3_test.shape[1])

y4_train_2 = y4_train.reshape(y4_train.shape[0], 1, y4_train.shape[1])
y4_val_2 = y4_val.reshape(y4_val.shape[0], 1, y4_val.shape[1])
y4_test_2 = y4_test.reshape(y4_test.shape[0], 1, y4_test.shape[1])

y5_train_2 = y5_train.reshape(y5_train.shape[0], 1, y5_train.shape[1])
y5_val_2 = y5_val.reshape(y5_val.shape[0], 1, y5_val.shape[1])
y5_test_2 = y5_test.reshape(y5_test.shape[0], 1, y5_test.shape[1])

y6_train_2 = y6_train.reshape(y6_train.shape[0], 1, y6_train.shape[1])
y6_val_2 = y6_val.reshape(y6_val.shape[0], 1, y6_val.shape[1])
y6_test_2 = y6_test.reshape(y6_test.shape[0], 1, y6_test.shape[1])

**NAESX**

In [ ]:
def build_lstm(hp):
  hp_units = hp.Int('units', min_value = 50, max_value = 500, step = 50)
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-1, 1e-2, 1e-3, 1e-4])
  hp_activation = hp.Choice('activation', values = ['relu', 'tanh', 'sigmoid'])
  model = tf.keras.Sequential()
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               input_shape = (X_train_2.shape[1],
                                              X_train_2.shape[2]),
                               return_sequences = True)))
  model.add(keras.layers.LSTM(units = hp_units, activation = hp_activation,
                             return_sequences = True))
  model.add(keras.layers.LSTM(units = hp_units, activation = hp_activation,
                              return_sequences = True))
  model.add(keras.layers.Dense(y1_train_2.shape[2]))

  model.compile(optimizer = Adam(learning_rate = hp_learning_rate),
                loss = 'mse',
                metrics = tf.keras.metrics.RootMeanSquaredError())
  return model

In [ ]:
lstm_tuner = kt.Hyperband(build_lstm,
                     objective = 'val_loss',
                     max_epochs = 10,
                     directory = 'keras_tuner_dir',
                     project_name = 'keras_lstm',
                     overwrite = True)

In [ ]:
### Early Stopping definition
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 5)

In [ ]:
### Hyperparameters search for 50 epochs
lstm_tuner.search(X_train_2, y1_train_2,
             epochs = 50, validation_data = (X_val_2, y1_val_2),
              callbacks = [stop_early])

Trial 30 Complete [00h 01m 41s]
val_loss: 9.58593882387504e-05

Best val_loss So Far: 9.290831803809851e-05
Total elapsed time: 00h 15m 14s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# Get the optimal hyperparameters
lstm_best_hps = lstm_tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"""
The hyperparameter search for NAESX LSTM is complete.
The optimal number of units in the first LSTM-connected
layer is {lstm_best_hps.get('units')},
the optimal activation is {lstm_best_hps.get('activation')}
and the optimal learning rate for the optimizer
is {lstm_best_hps.get('learning_rate')}
""")


The hyperparameter search for NAESX LSTM is complete.
The optimal number of units in the first LSTM-connected
layer is 100,
the optimal activation is tanh
and the optimal learning rate for the optimizer
is 0.0001



In [ ]:
### Find the optimal number of epochs to train the model with the hyperparameters
### obtained from the search.
### Build the model with the optimal hyperparameters and
### train it on the data for 100 epochs.
model = lstm_tuner.hypermodel.build(lstm_best_hps)

history = model.fit(X_train_2, y1_train_2,
                    epochs = 100, validation_data = (X_val_2, y1_val_2))

val_rmse_per_epoch = history.history['val_root_mean_squared_error']

best_epoch = val_rmse_per_epoch.index(min(val_rmse_per_epoch))

print('Best epoch: %d' % (best_epoch,))

Epoch 1/100
136/136 [==============================] - 9s 27ms/step - loss: 2.0875e-04 - root_mean_squared_error: 0.0144 - val_loss: 9.3492e-05 - val_root_mean_squared_error: 0.0097
Epoch 2/100
136/136 [==============================] - 1s 10ms/step - loss: 2.0799e-04 - root_mean_squared_error: 0.0144 - val_loss: 9.3725e-05 - val_root_mean_squared_error: 0.0097
Epoch 3/100
136/136 [==============================] - 1s 10ms/step - loss: 2.0773e-04 - root_mean_squared_error: 0.0144 - val_loss: 9.3895e-05 - val_root_mean_squared_error: 0.0097
Epoch 4/100
136/136 [==============================] - 1s 10ms/step - loss: 2.0745e-04 - root_mean_squared_error: 0.0144 - val_loss: 9.3956e-05 - val_root_mean_squared_error: 0.0097
Epoch 5/100
136/136 [==============================] - 1s 10ms/step - loss: 2.0739e-04 - root_mean_squared_error: 0.0144 - val_loss: 9.3169e-05 - val_root_mean_squared_error: 0.0097
Epoch 6/100
136/136 [==============================] - 1s 10ms/step - loss: 2.0743e-04 - r

In [ ]:
### Pass the optimal hyperparameters to hypermodel.
hypermodel_2 = lstm_tuner.hypermodel.build(lstm_best_hps)

### Retrain the model with the optimal number of epochs.

hypermodel_2.fit(X_train_2, y1_train_2,
                 epochs = best_epoch,
                 validation_data = (X_val_2, y1_val_2),
                 shuffle = False)

Epoch 1/9
136/136 [==============================] - 9s 20ms/step - loss: 2.0870e-04 - root_mean_squared_error: 0.0144 - val_loss: 9.4361e-05 - val_root_mean_squared_error: 0.0097
Epoch 2/9
136/136 [==============================] - 1s 10ms/step - loss: 2.0813e-04 - root_mean_squared_error: 0.0144 - val_loss: 9.4350e-05 - val_root_mean_squared_error: 0.0097
Epoch 3/9
136/136 [==============================] - 1s 10ms/step - loss: 2.0789e-04 - root_mean_squared_error: 0.0144 - val_loss: 9.4429e-05 - val_root_mean_squared_error: 0.0097
Epoch 4/9
136/136 [==============================] - 1s 10ms/step - loss: 2.0770e-04 - root_mean_squared_error: 0.0144 - val_loss: 9.4515e-05 - val_root_mean_squared_error: 0.0097
Epoch 5/9
136/136 [==============================] - 1s 9ms/step - loss: 2.0755e-04 - root_mean_squared_error: 0.0144 - val_loss: 9.4606e-05 - val_root_mean_squared_error: 0.0097
Epoch 6/9
136/136 [==============================] - 1s 10ms/step - loss: 2.0743e-04 - root_mean_squa

In [ ]:
### Predicted values using X_test_2.
y1_pred_lstm = hypermodel_2.predict(X_test_2)

In [ ]:
### Evaluate the hypermodel using the test sets.
eval_result_lstm = hypermodel_2.evaluate(X_test_2, y1_test_2)
print("[test loss, test mse]:", eval_result_lstm)

30/30 [==============================] - 0s 3ms/step - loss: 8.6780e-05 - root_mean_squared_error: 0.0093
[test loss, test mse]: [8.678028098074719e-05, 0.009315593168139458]


In [ ]:
y1_test_2 = y1_test_2.reshape((len(y1_test_2), 1))
y1_pred_lstm = y1_pred_lstm.reshape((len(y1_pred_lstm), 1))

In [ ]:
lstm_1_r2_score = r2_score(y1_test_2, y1_pred_lstm)
RMSE_LSTM_1 = np.sqrt(mean_squared_error(y1_test_2, y1_pred_lstm))
print('NAESX - R-squared LSTM: ' + str(lstm_1_r2_score))
print('NAESX - RMSE - LSTM: ' + str(RMSE_LSTM_1))

NAESX - R-squared LSTM: 0.0006741074319844831
NAESX - RMSE - LSTM: 0.009315592558266356


**VBMFX**

In [ ]:
def build_lstm(hp):
  hp_units = hp.Int('units', min_value = 50, max_value = 500, step = 50)
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-1, 1e-2, 1e-3, 1e-4])
  hp_activation = hp.Choice('activation', values = ['relu', 'tanh', 'sigmoid'])
  model = tf.keras.Sequential()
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               input_shape = (X_train_2.shape[1],
                                              X_train_2.shape[2]),
                               return_sequences = True)))
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               return_sequences = True)))
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               return_sequences = True)))  
  model.add(keras.layers.Dense(y2_train_2.shape[2]))

  model.compile(optimizer = Adam(learning_rate = hp_learning_rate),
                loss = 'mse',
                metrics = tf.keras.metrics.RootMeanSquaredError())
  return model

In [ ]:
lstm_tuner = kt.Hyperband(build_lstm,
                     objective = 'val_loss',
                     max_epochs = 10,
                     directory = 'keras_tuner_dir',
                     project_name = 'keras_lstm',
                     overwrite = True)

In [ ]:
### Early Stopping definition
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 5)

In [ ]:
### Hyperparameters search for 50 epochs
lstm_tuner.search(X_train_2, y2_train_2,
             epochs = 50, validation_data = (X_val_2, y2_val_2),
              callbacks = [stop_early])

Trial 30 Complete [00h 01m 33s]
val_loss: 5.890148258913541e-06

Best val_loss So Far: 5.783675533166388e-06
Total elapsed time: 00h 15m 29s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# Get the optimal hyperparameters
lstm_best_hps_2 = lstm_tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"""
The hyperparameter search for VBMFX LSTM is complete.
The optimal number of units in the LSTM-connected
layers is {lstm_best_hps_2.get('units')},
the optimal activation is {lstm_best_hps_2.get('activation')}
and the optimal learning rate for the optimizer
is {lstm_best_hps_2.get('learning_rate')}
""")


The hyperparameter search for VBMFX LSTM is complete.
The optimal number of units in the LSTM-connected
layers is 350,
the optimal activation is sigmoid
and the optimal learning rate for the optimizer
is 0.01



In [ ]:
### Find the optimal number of epochs to train the model with the hyperparameters
### obtained from the search.
### Build the model with the optimal hyperparameters and
### train it on the data for 100 epochs.
model = lstm_tuner.hypermodel.build(lstm_best_hps_2)

history = model.fit(X_train_2, y2_train_2,
                    epochs=100, validation_data = (X_val_2, y2_val_2))

val_rmse_per_epoch = history.history['val_root_mean_squared_error']

best_epoch = val_rmse_per_epoch.index(min(val_rmse_per_epoch))

print('Best epoch: %d' % (best_epoch,))

Epoch 1/100
136/136 [==============================] - 10s 45ms/step - loss: 0.2820 - root_mean_squared_error: 0.5311 - val_loss: 9.9032e-06 - val_root_mean_squared_error: 0.0031
Epoch 2/100
136/136 [==============================] - 5s 38ms/step - loss: 7.9523e-06 - root_mean_squared_error: 0.0028 - val_loss: 9.0031e-06 - val_root_mean_squared_error: 0.0030
Epoch 3/100
136/136 [==============================] - 5s 39ms/step - loss: 7.8564e-06 - root_mean_squared_error: 0.0028 - val_loss: 7.0023e-06 - val_root_mean_squared_error: 0.0026
Epoch 4/100
136/136 [==============================] - 5s 39ms/step - loss: 7.6231e-06 - root_mean_squared_error: 0.0028 - val_loss: 6.7293e-06 - val_root_mean_squared_error: 0.0026
Epoch 5/100
136/136 [==============================] - 5s 38ms/step - loss: 7.7034e-06 - root_mean_squared_error: 0.0028 - val_loss: 7.1311e-06 - val_root_mean_squared_error: 0.0027
Epoch 6/100
136/136 [==============================] - 5s 39ms/step - loss: 7.7626e-06 - root

In [ ]:
### Pass the optimal hyperparameters to hypermodel.
hypermodel_2 = lstm_tuner.hypermodel.build(lstm_best_hps_2)

### Retrain the model with the optimal number of epochs.

hypermodel_2.fit(X_train_2, y2_train_2,
                 epochs = best_epoch,
                 validation_data = (X_val_2, y2_val_2),
                 shuffle = False)

Epoch 1/98
136/136 [==============================] - 9s 41ms/step - loss: 0.3031 - root_mean_squared_error: 0.5505 - val_loss: 7.6564e-06 - val_root_mean_squared_error: 0.0028
Epoch 2/98
136/136 [==============================] - 5s 40ms/step - loss: 7.8041e-06 - root_mean_squared_error: 0.0028 - val_loss: 6.3030e-06 - val_root_mean_squared_error: 0.0025
Epoch 3/98
136/136 [==============================] - 7s 49ms/step - loss: 7.9256e-06 - root_mean_squared_error: 0.0028 - val_loss: 6.3838e-06 - val_root_mean_squared_error: 0.0025
Epoch 4/98
136/136 [==============================] - 5s 37ms/step - loss: 8.2420e-06 - root_mean_squared_error: 0.0029 - val_loss: 7.6418e-06 - val_root_mean_squared_error: 0.0028
Epoch 5/98
136/136 [==============================] - 5s 37ms/step - loss: 8.1615e-06 - root_mean_squared_error: 0.0029 - val_loss: 6.3925e-06 - val_root_mean_squared_error: 0.0025
Epoch 6/98
136/136 [==============================] - 5s 36ms/step - loss: 8.4276e-06 - root_mean_s

In [ ]:
### Predicted values using X_test_2.
y2_pred_lstm = hypermodel_2.predict(X_test_2)

In [ ]:
### Evaluate the hypermodel using the test sets.
eval_result_lstm = hypermodel_2.evaluate(X_test_2, y2_test_2)
print("[test loss, test mse]:", eval_result_lstm)

30/30 [==============================] - 1s 13ms/step - loss: 1.1534e-04 - root_mean_squared_error: 0.0107
[test loss, test mse]: [0.00011534138320712373, 0.010739710181951523]


In [ ]:
y2_test_2 = y2_test_2.reshape((len(y2_test_2), 1))
y2_pred_lstm = y2_pred_lstm.reshape((len(y2_pred_lstm), 1))

In [ ]:
lstm_2_r2_score = r2_score(y2_test_2, y2_pred_lstm)
RMSE_LSTM_2 = np.sqrt(mean_squared_error(y2_test_2, y2_pred_lstm))
print('VBMFX - R-squared - LSTM: ' + str(lstm_2_r2_score))
print('VBMFX - RMSE - LSTM: ' + str(RMSE_LSTM_2))

VBMFX - R-squared - LSTM: -28.31387327515966
VBMFX - RMSE - LSTM: 0.01073970971477074


**VEIEX**

In [ ]:
### Create LSTM model with the use of a function.
def build_lstm(hp):
  hp_units = hp.Int('units', min_value = 100, max_value = 500, step = 50)
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-1, 1e-2, 1e-3, 1e-4])
  hp_activation = hp.Choice('activation', values = ['relu', 'tanh', 'sigmoid'])
  model = tf.keras.Sequential()
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               input_shape = (X_train_2.shape[1],
                                              X_train_2.shape[2]),
                               return_sequences = True)))
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               return_sequences = True)))  
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               return_sequences = True)))
  model.add(keras.layers.Dense(y3_train_2.shape[2]))

  model.compile(optimizer = Adam(learning_rate = hp_learning_rate),
                loss = 'mse',
                metrics = tf.keras.metrics.RootMeanSquaredError())
  return model

In [ ]:
### Instantiate the tuner and perform hyperparameter tuning.
lstm_tuner = kt.Hyperband(build_lstm,
                     objective = 'val_loss',
                     max_epochs = 10,
                     directory = 'keras_tuner_dir',
                     project_name = 'keras_lstm',
                     overwrite = True)

In [ ]:
### Hyperparameters search for 5 epochs
lstm_tuner.search(X_train_2, y3_train_2,
             epochs = 50, validation_data = (X_val_2, y3_val_2),
              callbacks = [stop_early])

Trial 30 Complete [00h 00m 58s]
val_loss: 0.00011281395563855767

Best val_loss So Far: 0.0001107985372073017
Total elapsed time: 00h 12m 40s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# Get the optimal hyperparameters
lstm_best_hps_3 = lstm_tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"""
The hyperparameter search for VEIEX LSTM is complete.
The optimal number of units in the LSTM-connected
layers is {lstm_best_hps_3.get('units')},
the optimal activation is {lstm_best_hps_3.get('activation')}
and the optimal learning rate for the optimizer
is {lstm_best_hps_3.get('learning_rate')}
""")


The hyperparameter search for VEIEX LSTM is complete.
The optimal number of units in the LSTM-connected
layers is 150,
the optimal activation is tanh
and the optimal learning rate for the optimizer
is 0.0001



In [ ]:
### Find the optimal number of epochs to train the model with the hyperparameters
### obtained from the search.
### Build the model with the optimal hyperparameters and
### train it on the data for 100 epochs.
model = lstm_tuner.hypermodel.build(lstm_best_hps_3)

history = model.fit(X_train_2, y3_train_2,
                    epochs=100, validation_data = (X_val_2, y3_val_2))

val_rmse_per_epoch = history.history['val_root_mean_squared_error']

best_epoch = val_rmse_per_epoch.index(min(val_rmse_per_epoch))

print('Best epoch: %d' % (best_epoch,))

Epoch 1/100
136/136 [==============================] - 9s 25ms/step - loss: 2.0951e-04 - root_mean_squared_error: 0.0145 - val_loss: 1.1628e-04 - val_root_mean_squared_error: 0.0108
Epoch 2/100
136/136 [==============================] - 2s 13ms/step - loss: 2.0794e-04 - root_mean_squared_error: 0.0144 - val_loss: 1.1419e-04 - val_root_mean_squared_error: 0.0107
Epoch 3/100
136/136 [==============================] - 2s 13ms/step - loss: 2.0784e-04 - root_mean_squared_error: 0.0144 - val_loss: 1.1115e-04 - val_root_mean_squared_error: 0.0105
Epoch 4/100
136/136 [==============================] - 2s 13ms/step - loss: 2.0807e-04 - root_mean_squared_error: 0.0144 - val_loss: 1.1531e-04 - val_root_mean_squared_error: 0.0107
Epoch 5/100
136/136 [==============================] - 2s 13ms/step - loss: 2.0788e-04 - root_mean_squared_error: 0.0144 - val_loss: 1.1593e-04 - val_root_mean_squared_error: 0.0108
Epoch 6/100
136/136 [==============================] - 2s 14ms/step - loss: 2.0775e-04 - r

In [ ]:
### Pass the optimal hyperparameters to hypermodel.
hypermodel_2 = lstm_tuner.hypermodel.build(lstm_best_hps_3)

### Retrain the model with the optimal number of epochs.

hypermodel_2.fit(X_train_2, y3_train_2,
                 epochs = best_epoch,
                 validation_data = (X_val_2, y3_val_2),
                 shuffle = False)

Epoch 1/28
136/136 [==============================] - 12s 27ms/step - loss: 2.0899e-04 - root_mean_squared_error: 0.0145 - val_loss: 1.1334e-04 - val_root_mean_squared_error: 0.0106
Epoch 2/28
136/136 [==============================] - 2s 14ms/step - loss: 2.0837e-04 - root_mean_squared_error: 0.0144 - val_loss: 1.1335e-04 - val_root_mean_squared_error: 0.0106
Epoch 3/28
136/136 [==============================] - 2s 14ms/step - loss: 2.0811e-04 - root_mean_squared_error: 0.0144 - val_loss: 1.1344e-04 - val_root_mean_squared_error: 0.0107
Epoch 4/28
136/136 [==============================] - 2s 14ms/step - loss: 2.0792e-04 - root_mean_squared_error: 0.0144 - val_loss: 1.1348e-04 - val_root_mean_squared_error: 0.0107
Epoch 5/28
136/136 [==============================] - 2s 14ms/step - loss: 2.0777e-04 - root_mean_squared_error: 0.0144 - val_loss: 1.1350e-04 - val_root_mean_squared_error: 0.0107
Epoch 6/28
136/136 [==============================] - 2s 14ms/step - loss: 2.0767e-04 - root_m

In [ ]:
### Predicted values using X_test_2.
y3_pred_lstm = hypermodel_2.predict(X_test_2)

In [ ]:
y3_test_2 = y3_test_2.reshape((len(y3_test_2), 1))
y3_pred_lstm = y3_pred_lstm.reshape((len(y3_pred_lstm), 1))

In [ ]:
lstm_3_r2_score = r2_score(y3_test_2, y3_pred_lstm)
RMSE_LSTM_3 = np.sqrt(mean_squared_error(y3_test_2, y3_pred_lstm))
print('VEIEX - R-squared - LSTM: ' + str(lstm_3_r2_score))
print('VEIEX - RMSE - LSTM : ' + str(RMSE_LSTM_3))

VEIEX - R-squared - LSTM: 0.025920246253179124
VEIEX - RMSE - LSTM : 0.008378127131733883


**VFINX**

In [ ]:
### Create LSTM model with the use of a function.
def build_lstm(hp):
  hp_units = hp.Int('units', min_value = 50, max_value = 500, step = 50)
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-1, 1e-2, 1e-3, 1e-4])
  hp_activation = hp.Choice('activation', values = ['relu', 'tanh', 'sigmoid'])
  model = tf.keras.Sequential()
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               input_shape = (X_train_2.shape[1],
                                              X_train_2.shape[2]),
                               return_sequences = True)))
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               return_sequences = True)))
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               return_sequences = True)))
  model.add(keras.layers.Dense(y4_train_2.shape[2]))

  model.compile(optimizer = Adam(learning_rate = hp_learning_rate),
                loss = 'mse',
                metrics = tf.keras.metrics.RootMeanSquaredError())
  return model

In [ ]:
### Instantiate the tuner and perform hyperparameter tuning.
lstm_tuner = kt.Hyperband(build_lstm,
                     objective = 'val_loss',
                     max_epochs = 10,
                     directory = 'keras_tuner_dir',
                     project_name = 'keras_lstm',
                     overwrite = True)

In [ ]:
### Hyperparameters search for 5 epochs
lstm_tuner.search(X_train_2, y4_train_2,
             epochs = 50, validation_data = (X_val_2, y4_val_2),
              callbacks = [stop_early])

Trial 30 Complete [00h 00m 54s]
val_loss: 7.785183697706088e-05

Best val_loss So Far: 7.354691479122266e-05
Total elapsed time: 00h 12m 13s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# Get the optimal hyperparameters
lstm_best_hps_4 = lstm_tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"""
The hyperparameter search for VFINX LSTM is complete.
The optimal number of units in the first LSTM-connected
layer is {lstm_best_hps_4.get('units')},
the optimal activation is {lstm_best_hps_4.get('activation')}
and the optimal learning rate for the optimizer
is {lstm_best_hps_4.get('learning_rate')}
""")


The hyperparameter search for VFINX LSTM is complete.
The optimal number of units in the first LSTM-connected
layer is 100,
the optimal activation is tanh
and the optimal learning rate for the optimizer
is 0.001



In [ ]:
### Find the optimal number of epochs to train the model with the hyperparameters
### obtained from the search.
### Build the model with the optimal hyperparameters and
### train it on the data for 100 epochs.
model = lstm_tuner.hypermodel.build(lstm_best_hps_4)

history = model.fit(X_train_2, y4_train_2,
                    epochs = 100, validation_data = (X_val_2, y4_val_2))

val_rmse_per_epoch = history.history['val_root_mean_squared_error']

best_epoch = val_rmse_per_epoch.index(min(val_rmse_per_epoch))

print('Best epoch: %d' % (best_epoch,))

Epoch 1/100
136/136 [==============================] - 9s 20ms/step - loss: 1.4241e-04 - root_mean_squared_error: 0.0119 - val_loss: 7.4213e-05 - val_root_mean_squared_error: 0.0086
Epoch 2/100
136/136 [==============================] - 1s 10ms/step - loss: 1.3918e-04 - root_mean_squared_error: 0.0118 - val_loss: 7.5604e-05 - val_root_mean_squared_error: 0.0087
Epoch 3/100
136/136 [==============================] - 1s 10ms/step - loss: 1.3840e-04 - root_mean_squared_error: 0.0118 - val_loss: 7.7778e-05 - val_root_mean_squared_error: 0.0088
Epoch 4/100
136/136 [==============================] - 1s 10ms/step - loss: 1.3977e-04 - root_mean_squared_error: 0.0118 - val_loss: 7.4061e-05 - val_root_mean_squared_error: 0.0086
Epoch 5/100
136/136 [==============================] - 1s 10ms/step - loss: 1.3764e-04 - root_mean_squared_error: 0.0117 - val_loss: 7.6028e-05 - val_root_mean_squared_error: 0.0087
Epoch 6/100
136/136 [==============================] - 1s 10ms/step - loss: 1.3859e-04 - r

In [ ]:
### Pass the optimal hyperparameters to hypermodel.
hypermodel_2 = lstm_tuner.hypermodel.build(lstm_best_hps_4)

### Retrain the model with the optimal number of epochs.

hypermodel_2.fit(X_train_2, y4_train_2,
                 epochs = best_epoch,
                 validation_data = (X_val_2, y4_val_2),
                 shuffle = False)

Epoch 1/11
136/136 [==============================] - 10s 33ms/step - loss: 1.4679e-04 - root_mean_squared_error: 0.0121 - val_loss: 7.5888e-05 - val_root_mean_squared_error: 0.0087
Epoch 2/11
136/136 [==============================] - 1s 11ms/step - loss: 1.4093e-04 - root_mean_squared_error: 0.0119 - val_loss: 7.8801e-05 - val_root_mean_squared_error: 0.0089
Epoch 3/11
136/136 [==============================] - 1s 11ms/step - loss: 1.4007e-04 - root_mean_squared_error: 0.0118 - val_loss: 7.6305e-05 - val_root_mean_squared_error: 0.0087
Epoch 4/11
136/136 [==============================] - 1s 10ms/step - loss: 1.3976e-04 - root_mean_squared_error: 0.0118 - val_loss: 7.6009e-05 - val_root_mean_squared_error: 0.0087
Epoch 5/11
136/136 [==============================] - 1s 10ms/step - loss: 1.3954e-04 - root_mean_squared_error: 0.0118 - val_loss: 7.5255e-05 - val_root_mean_squared_error: 0.0087
Epoch 6/11
136/136 [==============================] - 1s 10ms/step - loss: 1.3929e-04 - root_m

In [ ]:
### Predicted values using X_test_2.
y4_pred_lstm = hypermodel_2.predict(X_test_2)

In [ ]:
y4_test_2 = y4_test_2.reshape((len(y4_test_2), 1))
y4_pred_lstm = y4_pred_lstm.reshape((len(y4_pred_lstm), 1))

In [ ]:
lstm_4_r2_score = r2_score(y4_test_2, y4_pred_lstm)
RMSE_LSTM_4 = np.sqrt(mean_squared_error(y4_test_2, y4_pred_lstm))
print('VFINX - R-squared - LSTM :' + str(lstm_4_r2_score))
print('VFINX - RMSE - LSTM: ' + str(RMSE_LSTM_4))

VFINX - R-squared - LSTM :0.024867185861314867
VFINX - RMSE - LSTM: 0.008554045121884143


**VGSLX**

In [ ]:
### Create LSTM model with the use of a function.
def build_lstm(hp):
  hp_units = hp.Int('units', min_value = 50, max_value = 500, step = 50)
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-1, 1e-2, 1e-3, 1e-4])
  hp_activation = hp.Choice('activation', values = ['relu', 'tanh', 'sigmoid'])
  model = tf.keras.Sequential()
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               input_shape = (X_train_2.shape[1],
                                              X_train_2.shape[2]),
                               return_sequences = True)))
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               return_sequences = True)))
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               return_sequences = True)))  
  model.add(keras.layers.Dense(y5_train_2.shape[2]))

  model.compile(optimizer = Adam(learning_rate = hp_learning_rate),
                loss = 'mse',
                metrics = tf.keras.metrics.RootMeanSquaredError())
  return model

In [ ]:
### Instantiate the tuner and perform hyperparameter tuning.
lstm_tuner = kt.Hyperband(build_lstm,
                     objective = 'val_loss',
                     max_epochs = 10,
                     directory = 'keras_tuner_dir',
                     project_name = 'keras_lstm',
                     overwrite = True)

In [ ]:
### Hyperparameters search for 5 epochs
lstm_tuner.search(X_train_2, y5_train_2,
             epochs = 50, validation_data = (X_val_2, y5_val_2),
              callbacks = [stop_early])

Trial 30 Complete [00h 01m 13s]
val_loss: 0.00010601565008983016

Best val_loss So Far: 0.00010580759408185259
Total elapsed time: 00h 13m 28s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# Get the optimal hyperparameters
lstm_best_hps_5 = lstm_tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"""
The hyperparameter search for LSTM is complete.
The optimal number of units in the LSTM-connected
layers is {lstm_best_hps_5.get('units')},
the optimal activation is {lstm_best_hps_5.get('activation')}
and the optimal learning rate for the optimizer
is {lstm_best_hps_5.get('learning_rate')}
""")


The hyperparameter search for LSTM is complete.
The optimal number of units in the LSTM-connected
layers is 300,
the optimal activation is sigmoid
and the optimal learning rate for the optimizer
is 0.1



In [ ]:
### Find the optimal number of epochs to train the model with the hyperparameters
### obtained from the search.
### Build the model with the optimal hyperparameters and
### train it on the data for 100 epochs.
model = lstm_tuner.hypermodel.build(lstm_best_hps_5)

history = model.fit(X_train_2, y5_train_2,
                    epochs=100, validation_data = (X_val_2, y5_val_2))

val_rmse_per_epoch = history.history['val_root_mean_squared_error']

best_epoch = val_rmse_per_epoch.index(min(val_rmse_per_epoch))

print('Best epoch: %d' % (best_epoch,))

Epoch 1/100
136/136 [==============================] - 9s 36ms/step - loss: 8.1375 - root_mean_squared_error: 2.8526 - val_loss: 1.0861e-04 - val_root_mean_squared_error: 0.0104
Epoch 2/100
136/136 [==============================] - 4s 30ms/step - loss: 4.6482e-04 - root_mean_squared_error: 0.0216 - val_loss: 1.1184e-04 - val_root_mean_squared_error: 0.0106
Epoch 3/100
136/136 [==============================] - 4s 33ms/step - loss: 4.6493e-04 - root_mean_squared_error: 0.0216 - val_loss: 1.1436e-04 - val_root_mean_squared_error: 0.0107
Epoch 4/100
136/136 [==============================] - 4s 32ms/step - loss: 4.8049e-04 - root_mean_squared_error: 0.0219 - val_loss: 1.9072e-04 - val_root_mean_squared_error: 0.0138
Epoch 5/100
136/136 [==============================] - 4s 32ms/step - loss: 4.9581e-04 - root_mean_squared_error: 0.0223 - val_loss: 1.3566e-04 - val_root_mean_squared_error: 0.0116
Epoch 6/100
136/136 [==============================] - 4s 32ms/step - loss: 4.8799e-04 - root_

In [ ]:
### Pass the optimal hyperparameters to hypermodel.
hypermodel_2 = lstm_tuner.hypermodel.build(lstm_best_hps_5)

### Retrain the model with the optimal number of epochs.

hypermodel_2.fit(X_train_2, y5_train_2,
                 epochs = best_epoch,
                 validation_data = (X_val_2, y5_val_2),
                 shuffle = False)

Epoch 1/26
136/136 [==============================] - 8s 33ms/step - loss: 7.8693 - root_mean_squared_error: 2.8052 - val_loss: 1.7623e-04 - val_root_mean_squared_error: 0.0133
Epoch 2/26
136/136 [==============================] - 4s 28ms/step - loss: 4.5234e-04 - root_mean_squared_error: 0.0213 - val_loss: 1.0743e-04 - val_root_mean_squared_error: 0.0104
Epoch 3/26
136/136 [==============================] - 4s 29ms/step - loss: 4.5261e-04 - root_mean_squared_error: 0.0213 - val_loss: 1.1054e-04 - val_root_mean_squared_error: 0.0105
Epoch 4/26
136/136 [==============================] - 4s 29ms/step - loss: 4.6023e-04 - root_mean_squared_error: 0.0215 - val_loss: 1.1292e-04 - val_root_mean_squared_error: 0.0106
Epoch 5/26
136/136 [==============================] - 4s 27ms/step - loss: 4.8279e-04 - root_mean_squared_error: 0.0220 - val_loss: 1.0590e-04 - val_root_mean_squared_error: 0.0103
Epoch 6/26
136/136 [==============================] - 4s 28ms/step - loss: 5.0340e-04 - root_mean_s

In [ ]:
### Predicted values using X_test_2.
y5_pred_lstm = hypermodel_2.predict(X_test_2)

In [ ]:
y5_test_2 = y5_test_2.reshape((len(y5_test_2), 1))
y5_pred_lstm = y5_pred_lstm.reshape((len(y5_pred_lstm), 1))

In [ ]:
lstm_5_r2_score = r2_score(y5_test_2, y5_pred_lstm)
RMSE_LSTM_5 = np.sqrt(mean_squared_error(y5_test_2, y5_pred_lstm))
print('VGSLX - R-squared - LSTM: ' + str(lstm_5_r2_score))
print('VGSLX - RMSE - LSTM: ' + str(RMSE_LSTM_5))

VGSLX - R-squared - LSTM: -1.663632465670414
VGSLX - RMSE - LSTM: 0.01348117230515925


**VGTSX**

In [ ]:
### Create LSTM model with the use of a function.
def build_lstm(hp):
  hp_units = hp.Int('units', min_value = 50, max_value = 500, step = 50)
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-1, 1e-2, 1e-3, 1e-4])
  hp_activation = hp.Choice('activation', values = ['relu', 'tanh', 'sigmoid'])
  model = tf.keras.Sequential()
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               input_shape = (X_train_2.shape[1],
                                              X_train_2.shape[2]),
                               return_sequences = True)))
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               return_sequences = True)))  
  model.add((keras.layers.LSTM(units = hp_units,
                               activation = hp_activation,
                               return_sequences = True)))
  model.add(keras.layers.Dense(y6_train_2.shape[2]))

  model.compile(optimizer = Adam(learning_rate = hp_learning_rate),
                loss = 'mse',
                metrics = tf.keras.metrics.RootMeanSquaredError())
  return model

In [ ]:
### Instantiate the tuner and perform hyperparameter tuning.
lstm_tuner = kt.Hyperband(build_lstm,
                     objective = 'val_loss',
                     max_epochs = 10,
                     directory = 'keras_tuner_dir',
                     project_name = 'keras_lstm',
                     overwrite = True)

In [ ]:
### Hyperparameters search for 5 epochs
lstm_tuner.search(X_train_2, y6_train_2,
             epochs = 50, validation_data = (X_val_2, y6_val_2),
              callbacks = [stop_early])

Trial 30 Complete [00h 00m 33s]
val_loss: 9.98601972241886e-05

Best val_loss So Far: 9.98601972241886e-05
Total elapsed time: 00h 11m 16s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# Get the optimal hyperparameters
lstm_best_hps_6 = lstm_tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"""
The hyperparameter search for VGTSX LSTM is complete.
The optimal number of units in the LSTM-connected
layers is {lstm_best_hps_6.get('units')},
the optimal activation is {lstm_best_hps_6.get('activation')}
and the optimal learning rate for the optimizer
is {lstm_best_hps_6.get('learning_rate')}
""")


The hyperparameter search for VGTSX LSTM is complete.
The optimal number of units in the LSTM-connected
layers is 250,
the optimal activation is relu
and the optimal learning rate for the optimizer
is 0.001



In [ ]:
### Find the optimal number of epochs to train the model with the hyperparameters
### obtained from the search.
### Build the model with the optimal hyperparameters and
### train it on the data for 100 epochs.
model = lstm_tuner.hypermodel.build(lstm_best_hps_6)

history = model.fit(X_train_2, y6_train_2,
                    epochs=100, validation_data = (X_val_2, y6_val_2))

val_rmse_per_epoch = history.history['val_root_mean_squared_error']

best_epoch = val_rmse_per_epoch.index(min(val_rmse_per_epoch))

print('Best epoch: %d' % (best_epoch,))

Epoch 1/100
136/136 [==============================] - 8s 28ms/step - loss: 1.6679e-04 - root_mean_squared_error: 0.0129 - val_loss: 1.2495e-04 - val_root_mean_squared_error: 0.0112
Epoch 2/100
136/136 [==============================] - 3s 25ms/step - loss: 1.6333e-04 - root_mean_squared_error: 0.0128 - val_loss: 1.0150e-04 - val_root_mean_squared_error: 0.0101
Epoch 3/100
136/136 [==============================] - 3s 25ms/step - loss: 1.6577e-04 - root_mean_squared_error: 0.0129 - val_loss: 1.0267e-04 - val_root_mean_squared_error: 0.0101
Epoch 4/100
136/136 [==============================] - 3s 25ms/step - loss: 1.6456e-04 - root_mean_squared_error: 0.0128 - val_loss: 1.0185e-04 - val_root_mean_squared_error: 0.0101
Epoch 5/100
136/136 [==============================] - 3s 25ms/step - loss: 1.6495e-04 - root_mean_squared_error: 0.0128 - val_loss: 1.0167e-04 - val_root_mean_squared_error: 0.0101
Epoch 6/100
136/136 [==============================] - 3s 24ms/step - loss: 1.6461e-04 - r

In [ ]:
### Pass the optimal hyperparameters to hypermodel.
hypermodel_2 = lstm_tuner.hypermodel.build(lstm_best_hps_6)

### Retrain the model with the optimal number of epochs.

hypermodel_2.fit(X_train_2, y6_train_2,
                 epochs = best_epoch,
                 validation_data = (X_val_2, y6_val_2),
                 shuffle = False)

Epoch 1/50
136/136 [==============================] - 8s 29ms/step - loss: 1.6684e-04 - root_mean_squared_error: 0.0129 - val_loss: 1.0714e-04 - val_root_mean_squared_error: 0.0104
Epoch 2/50
136/136 [==============================] - 3s 24ms/step - loss: 1.6481e-04 - root_mean_squared_error: 0.0128 - val_loss: 1.0167e-04 - val_root_mean_squared_error: 0.0101
Epoch 3/50
136/136 [==============================] - 3s 24ms/step - loss: 1.6642e-04 - root_mean_squared_error: 0.0129 - val_loss: 9.9336e-05 - val_root_mean_squared_error: 0.0100
Epoch 4/50
136/136 [==============================] - 3s 24ms/step - loss: 1.6517e-04 - root_mean_squared_error: 0.0129 - val_loss: 9.9446e-05 - val_root_mean_squared_error: 0.0100
Epoch 5/50
136/136 [==============================] - 3s 24ms/step - loss: 1.6502e-04 - root_mean_squared_error: 0.0128 - val_loss: 9.9402e-05 - val_root_mean_squared_error: 0.0100
Epoch 6/50
136/136 [==============================] - 3s 24ms/step - loss: 1.6482e-04 - root_me

In [ ]:
### Predicted values using X_test_2.
y6_pred_lstm = hypermodel_2.predict(X_test_2)

In [ ]:
y6_test_2 = y6_test_2.reshape((len(y6_test_2), 1))
y6_pred_lstm = y6_pred_lstm.reshape((len(y6_pred_lstm), 1))

In [ ]:
lstm_6_r2_score = r2_score(y6_test_2, y6_pred_lstm)
RMSE_LSTM_6 = np.sqrt(mean_squared_error(y6_test_2, y6_pred_lstm))
print('VGTSX - R-squared - LSTM: ' + str(lstm_6_r2_score))
print('VGTSX - RMSE - LSTM: ' + str(RMSE_LSTM_6))

VGTSX - R-squared - LSTM: -0.20097406233044635
VGTSX - RMSE - LSTM: 0.007750273486272318
